# Monitoring of *Gracilaria vermiculophylla*: Mapping its Distribution

at the Site of its First European Observation

Simon Oiry¹ [](https://orcid.org/0000-0001-7161-5246) (, Institut des Substances et Organismes de la Mer, ISOMer, , , )  
Bede Ffinian Rowe Davies¹ [](https://orcid.org/0000-0001-6462-4347) ()  
Pierre Gernez¹ [](https://orcid.org/0000-0003-2055-410X) ()  
Laurent Barillé¹ [](https://orcid.org/0000-0001-5138-2684) ()  
December 21, 2024

The invasive red macroalga *Gracilaria vermiculophylla* has significantly impacted intertidal ecosystems in temperate estuaries globally. This study utilized drone-based multispectral remote sensing to map the spatial and temporal distribution of *G. vermiculophylla* in its first documented European site, the Bélon Estuary, alongside additional sites in Spain and France. By adapting the neural network classification model DISCOV, trained with a comprehensive dataset, we achieved 91.1% accuracy in distinguishing *G. vermiculophylla* from other macroalgal taxa. Historical aerial imagery revealed a progressive expansion of *G. vermiculophylla* from its initial appearance in 1976, approximately 20 years before its first description in the literature, to extensive colonization by 2024. Concurrent LiDAR data enabled precise characterization of intertidal topography, demonstrating a strong association between algal cover, elevation, and slope. Dense mats were consistently observed in flat, elevated mudflat areas, with reduced presence in steeper or lower zones. These patterns highlight the species’ preference for stable sedimentary environments with reduced hydrodynamic forces. Temporal analyses also linked its spread to anthropogenic activities, notably aquaculture. Our findings emphasize the utility of high-resolution drone imaging for invasive species monitoring and habitat mapping, offering critical insights into the ecological dynamics of *G. vermiculophylla* and its drivers. This scalable method facilitates proactive management strategies by enabling early detection and detailed assessment of invasion patterns. The integration of remote sensing and in situ validation establishes a robust framework for ecological monitoring, contributing to the understanding of biological invasions and their environmental consequences. This approach can inform management interventions to mitigate the impacts of *G. vermiculophylla* and similar invasive species.

[1]

[1] Institut des Substances et Organismes de la Mer, ISOMer, Nantes Université, UR 2160, F-44000 Nantes, France

In [ ]:
library(tidyverse)
library(terra)
library(rnaturalearth) 
library(rnaturalearthdata) 
library(sf)
library(Utilities.Package)
library(flextable)


Attachement du package : 'flextable'

Les objets suivants sont masqués depuis 'package:terra':

    align, colorize, rotate, width

L'objet suivant est masqué depuis 'package:purrr':

    compose

In [ ]:
my_comma<-scales::label_comma(accuracy = NULL, big.mark = ",",decimal.mark = ".")

In [ ]:
library(fs)

# Define the size threshold in bytes (100 MB)
size_threshold <- 100 * 1024 * 1024

# Get a list of all files in the repository
files <- dir_ls(recurse = TRUE, type = "file")

# Filter files larger than the size threshold
large_files <- files[file_info(files)$size > size_threshold]

# Check if there are large files
if (length(large_files) > 0) {
  # Read the existing .gitignore file if it exists
  gitignore_path <- ".gitignore"
  gitignore_content <- if (file_exists(gitignore_path)) {
    readLines(gitignore_path)
  } else {
    character(0)
  }
  
  # Identify files not already in .gitignore
  files_to_add <- large_files[!large_files %in% gitignore_content]
  
  # Append new large files to .gitignore
  if (length(files_to_add) > 0) {
    writeLines(c(gitignore_content, files_to_add), gitignore_path)
    message(length(files_to_add), " file(s) added to .gitignore.")
  } else {
    message("No new files to add to .gitignore.")
  }
} else {
  message("No files larger than 100 MB found.")
}

# 1. Title proposition

-   Monitoring the marine invasive alien species *Gracilaria vermiculophylla* remotely

# 2. Introduction

The introduction of Non-Indigenous Species (NIS) in terrestrial, freshwater, and marine ecosystems is one of the major threats to biodiversity worldwide. In particular, the proliferation and rapid spread of Invasive Alien Species (IAS) can radically change the structure and functioning of marine ecosystems, requiring effective assessment and monitoring programs ([Massé et al., 2023](#ref-d15020161)). In Europe, 874 NIS have been introduced to the marine environment so far (i.e. until 2020) and it is expected that the rate of biological invasions will continue to increase in the coming years ([Zenetos et al., 2022](#ref-d14121077)). Macroalgae represent more than 40 % of the NIS introduced to Europe waters, with many species native to the Temperate Northern Pacific ([Williams and Smith, 2007](#ref-williams2007global)).

Amongst all invasive macroalgae, *Gracilaria vermiculophylla* ([Papenfuss, 1967](#ref-WoRMS303450)) (original name *Gracilariopsis vermiculophylla* ([OHMI, 1956](#ref-ohmi1956contributions)); also known as *Agarophyton vermiculophyllum* ([Gurgel et al., 2018](#ref-gurgel2018systematics))), has spread extensively from its native distribution range in Japan and Korea ([Terada and Yamamoto, 2002](#ref-terada2002review)). This spread has occurred across temperate estuaries in North America, Europe, and other regions, facilitated by aquaculture and maritime activities ([Krueger-Hadfield et al., 2017](#ref-krueger2017genetic); [Rueness, 2005](#ref-rueness2005life); [Weinberger et al., 2008](#ref-weinberger2008invasive)).

In regions like the Baltic Sea and the eastern United States, it can affect native fucoid macroalgae and seagrasses negatively ([Firth et al., 2024](#ref-firth2024invasive); [Thomsen et al., 2013](#ref-thomsen2013effects); [Van Katwijk, 2003](#ref-van2003reintroduction)). It can also alter sediment composition ([Nyberg et al., 2009](#ref-nyberg2009flora)), and disrupts trophic interactions ([Ginneken et al., 2018](#ref-van2018global)). However *G. vermiculophylla* create new habitats for invertebrates and juvenile fish in a soft-bottom environment ([Davoult et al., 2017](#ref-davoult2017multiple)) and, more generally, can positively enhance ecosystem processes ([Ramus et al., 2017](#ref-ramus2017invasive)). The negative and positive effects of this species ([Thomsen et al., 2009](#ref-thomsen2009distribution)), which now dominate some coastal ecosystems, underscore the importance of monitoring and managing its population, particularly as climate change and anthropogenic pressures continue to facilitate biological invasions. *G. vermiculophylla* success as an invader stems from its tolerance to a wide range of environmental conditions, including temperature ([Sotka et al., 2018](#ref-sotka2018combining)), nutrient variability ([Abreu et al., 2011](#ref-abreu2011nitrogen)), and salinity ([Weinberger et al., 2008](#ref-weinberger2008invasive)). Its growth capacity at low salinities ([Nyberg, 2007](#ref-nyberg2007introduced); [Rueness, 2005](#ref-rueness2005life)) explains its presence in the brackish waters of the Baltic Sea ([Weinberger et al., 2008](#ref-weinberger2008invasive)) but also in the mesohaline sheltered part of estuaries of the Atlantic coast of Europe **(Surget et al., 2017)**. It is also present in confined areas of lagoons characterized by low hydrodynamism ([Abreu et al., 2011](#ref-abreu2011nitrogen); [Sfriso et al., 2012](#ref-sfriso2012spreading)). In Europe, it was first observed in 1996 in the Belon estuary (France) and later in many other estuaries on the Brittany coast of France ([Rueness, 2005](#ref-rueness2005life)). It can be found on hard substrates such as invertebrate’s tubes and shells providing a substratum ([Thomsen et al., 2007](#ref-thomsen2007gracilaria)) or attached to pebbles and rocks ([Terada and Yamamoto, 2002](#ref-terada2002review)) but the largest populations are colonizing soft-bottom sediment and particularly estuarine intertidal mudflats **(Surget et al., 2017)**. In this habitat, extensive dark red mats are observed at low tide, covering vast areas that have largely been unquantified in most studies. Therefore, *G. vermiculophylla* can establish populations in soft-bottom sediment habitats, previously devoid of macroalgae ([Ramus et al., 2017](#ref-ramus2017invasive)). These mats are usually monospecific, with the alga thalli partially buried in the mud ([Rueness, 2005](#ref-rueness2005life); [Surget, 2017](#ref-surget2017processus)). Intertidal mats can, however, be temporarily overgrown by ephemeral green macroalgae ([Weinberger et al., 2008](#ref-weinberger2008invasive)). In the European estuaries where *G. vermiculophylla* was first documented, large monospecific mats were reported to be confined to the upper intertidal zones ([Rueness, 2005](#ref-rueness2005life)); however, their spatial distribution relative to the mudflat topography and elevation had not been quantitatively assessed. In coastal lagoons of the East Atlantic coast, Besterman et al. ([2021](#ref-besterman2021predicting)) have shown that the mudflat topography was a significant predictor of its abundance. In fact, *G. vermiculophylla* has never been mapped using remote sensing techniques, and existing descriptions of its distribution lack spatially explicit mapping ([Abreu et al., 2011](#ref-abreu2011nitrogen); [Sfriso et al., 2012](#ref-sfriso2012spreading); [Thomsen et al., 2007](#ref-thomsen2007gracilaria); [Weinberger et al., 2008](#ref-weinberger2008invasive)).

Remote sensing has revolutionized our ability to monitor and manage coastal ecosystems, offering efficient and scalable methods for detecting environmental changes in intertidal vegetation across a wide range of spatio-temporal scales ([Calleja et al., 2017](#ref-calleja2017long); [Davies et al., 2024a](#ref-davies2024intertidal), [2024b](#ref-davies2024sentinel); [Valle et al., 2015](#ref-valle2015mapping); [Zoffoli et al., 2021](#ref-zoffoli2021decadal)). Among remote-sensing technologies, drone-based imagery has recently emerged as a particularly promising tool for studying the spatial distribution of intertidal primary producers such as benthic microalgae ([Román et al., 2024](#ref-roman2024mapping), [2021](#ref-roman2021using)), seagrass ([Chand and Bollard, 2021](#ref-chand2021low); [Duffy et al., 2018](#ref-duffy2018spatial); [Román et al., 2021](#ref-roman2021using)) and macroalgae ([Diruit et al., 2022](#ref-rs14133124); [Peidro-Devesa et al., 2024](#ref-peidro2024quantifying)). While it lacks the temporal consistency of satellite missions, drone remote sensing makes it possible to acquire at extremely high spatial resolution (i.e. cm-scale), rapidly target specific areas of interest, and provide observations in overcast conditions. In particular, the potential of drone remote sensing for monitoring the surface area occupied by IAS has been demonstrated ([Roca et al., 2022](#ref-roca2022monitoring)). Drone-based photogrammetry also makes it possible to characterize the distribution of intertidal vegetation together with mudflat geomorphology, thus improving our understanding of primary producers patterning ([Brunier et al., 2022](#ref-brunier2022evolution); [Douglas et al., 2024](#ref-douglas2024linking)).

This study applied a drone-based remote sensing approach to map *G. vermiculophylla* spatial distribution at a very-high spatial resolution (centimeter) in intertidal estuaries of European Atlantic coast. We adapted a neural network classification model, Drone Intertidal Substrate Classification Of Vegetation (DISCOV, ([Oiry et al., 2024](#ref-rs16234383))) by re-training the model with new pixels of *G. vermiculophylla*. An *in situ* data validation dataset was obtained from Franch and Spanish sites to estimate the classification accuracy. LIDAR data were concurrently acquired to map the intertidal elevation accurately. A Generalized Linear Mixed Effect Model (GLMM) was used to examine the relationship between the seaweed spatial distribution and spatial metrics quantifying the mudflat topography. We expected the presence of *G. vermiculophylla* in mudflats to be associated with a specific height range as well as being more closely related to flat areas of the intertidal zone. In the Belon estuary (South Brittany, France) where it was first observed in Europe, a time series, starting from 1952, of RGB images was analysed to describe the temporal changes of its distribution over the last seventy years.

# 3. Materiel & Methods

In [ ]:
building <- T   

layout<-"Data/Workflow_Figure" %>%
  list.files("V2.txt",full.names = T, recursive = T) %>%
  read_delim(delim = "\t", escape_double = FALSE,
    trim_ws = TRUE) %>%
  dplyr::filter(Evaluate == T)


layout$Text<- gsub("alaligne","\n", layout$Text)

boxes<- layout %>% 
  dplyr::filter(Type == "Box",
                shape == "square")


text<- layout %>% 
  dplyr::filter(Type == "Box")

diamonds<- layout %>% 
  dplyr::filter(Type == "Box",
                shape == "diamond")
diff = 0.1
for (i in 1:nrow(diamonds)) {
  diam_a<-diamonds[i,]
  
df<-data.frame(  
  x = c(diam_a$xmin - diff, diam_a$xmin + diff, diam_a$xmax + diff, diam_a$xmax - diff),
  y = c(diam_a$ymin, diam_a$ymax, diam_a$ymax, diam_a$ymin),
  Type = rep(diam_a$Type,4),
  Text = rep(diam_a$Text,4),
  Color = rep(diam_a$color,4),
  Evaluate = rep(diam_a$Evaluate,4),
  Resolution =  rep(diam_a$Resolution,4),
  Evaluate_bigbox = rep(diam_a$Evaluate_bigbox,4),
  alpha = rep(diam_a$alpha,4),
  shape =  rep(diam_a$shape,4)
)
  if(i == 1){
    diamond_list<-list(df)
  }else{
    diamond_list[[i]]<-df
  }
}



Arrows<- layout %>% 
  dplyr::filter(Type == "Arrow",
                Evaluate == T)

Group_box<-layout %>% 
  dplyr::filter(Type == "Box",
                Evaluate_bigbox == T) %>% 
  group_by(Group) %>% 
  dplyr::summarise(xmin = min(xmin)-0.1,
                   xmax = max(xmax)+0.1,
                   ymin = min(ymin)-0.1,
                   ymax = max(ymax)+0.1,
                   alpha = 0.1,
                   color = color)

resolution_box<-layout %>% 
  dplyr::filter(Type == "Box",
                Resolution != "NA") %>% 
  group_by(Resolution) %>% 
  dplyr::summarise(xmin = min(xmin)-0.1,
                   xmax = max(xmax)+0.1,
                   ymin = min(ymin)-0.1,
                   ymax = max(ymax)+0.1,
                   alpha = 0.1,
                   color = color)

legend = data.frame(color = unique(layout$color)) %>% 
  dplyr::filter(!is.na(color)) %>% 
  mutate(label = case_when(color == "#EA6155" ~ "Input Data",
                           color == "#DC9614" ~ "Pre-processing",
                           color == "#52A4DA" ~ "Model Building",
                           color == "#79DA52" ~ "Validation",
                           color == "#DA9652" ~ "Output Data",
                           color == "darkcyan" ~ "Final Product",
                           color == "#DA5273"  ~ "Prediction",
                           TRUE ~ "NA")) %>% 
  arrange

segment<-layout %>% 
  dplyr::filter(Type == "Segment")

if(building == T){
  
text_size = 6 #### 6 to plot it fullscreen in R,  1.8 for exporting with ggsave
line_size =1 ####arrow_size 0.3 to export  1 to for R plotting

legend_size = 1 #### 0.3 to export, 1 for R ploting
legend_text = 10 #### 4 to export, 10 for R ploting
}else{
  
text_size = 1.85 #### 6 to plot it fullscreen in R,  1.8 for exporting with ggsave
line_size =0.3 ####arrow_size 0.3 to export  1 to for R plotting

legend_size = 0.3 #### 0.3 to export, 1 for R ploting
legend_text = 5 #### 5 to export, 10 for R ploting
  
}


a<-ggplot()+
  
  # statebins:::geom_rrect(data = Group_box,
  #           mapping = aes(xmin = xmin,
  #                         xmax= xmax,
  #                         ymin = ymin,
  #                         ymax = ymax),
  #           alpha = 0.05,
  #           size = 0.2,
  #           show.legend = F,
  #           fill = Group_box$color,
  #           color = "black")+
  
    # statebins:::geom_rrect(data = resolution_box,
    #         mapping = aes(xmin = xmin,
    #                       xmax= xmax,
    #                       ymin = ymin,
    #                       ymax = ymax,
    #                       alpha = rep(c("Input Data", "Pre-processing", "Model Building", "Prediction", "Validation", "Output Data", "Final Product"),2)),
    #         fill = "NA",
    #         linetype = 2,
    #         size = 0.4,
    #         color = "black")+
  
  scale_alpha_manual(name = "", 
                     values = c(1,1,1,1,1,1,1),
                     labels = c("Input Data", "Pre-Processing", "Model Building", "Prediction", "Validation", "Output Data", "Final Product"),
                     guide = guide_legend(ncol = 1, override.aes = list(fill = c("#f94144", "#f3722c", "#f8961e", "#f9c74f", "#90be6d", "#43aa8b", "#577590"),
                                                                        shape = c(22),
                                                                        linetype = c(1),
                                                                        alpha = c(0.5)
                                                                        )
                                          )
                     )+
  
  statebins:::geom_rrect(data = boxes,
            mapping = aes(xmin = xmin,
                          xmax= xmax,
                          ymin = ymin,
                          ymax = ymax
                          ),
            size = 0.2,
            alpha = as.numeric(boxes$alpha),
            fill = boxes$color,
            color = "black")
  
  for(i in 1:length(diamond_list)){
    poly_i<-diamond_list[[i]]
    a<-a+
      geom_polygon(data = poly_i, 
                   mapping = aes(x = x, 
                                 y = y
                                 ),
                  size = 0.2,
                   alpha = poly_i$alpha,
                   fill = poly_i$Color,
                   color = "black")
    
  }

if(building == T){
    a<-a+
    geom_text(data = text,
            mapping = aes(x = (xmax+xmin)/2, y = (ymax + ymin)/2, label = (ID)), size = text_size)
  
}else{
    a<-a+
    geom_text(data = text,
            mapping = aes(x = (xmax+xmin)/2, y = (ymax + ymin)/2, label = (Text)), size = text_size)
}


  a<-a+
    geom_segment(data = Arrows, aes( x = xmin, xend = xmax, y = ymin, yend = ymax),
    size=line_size, linejoin = "mitre", lineend = "butt",
    arrow = arrow(length = unit(1, "mm"), type= "closed")
    )+
    geom_segment(data = segment, aes( x = xmin, xend = xmax, y = ymin, yend = ymax),
    size=line_size, linejoin = "mitre", lineend = "square"
    )
    
  if(building == F){
  a<- a+ theme_void()
  }
  
  a<-a+
  theme(legend.position = c(0.88,0.3),
        legend.text = element_text(size = legend_text),
        legend.key.size = unit(legend_size, 'cm'),
        plot.background = element_rect(fill = "white",color = NA)
        )+
  scale_x_continuous(breaks = scales::breaks_width(0.5))+
  scale_y_continuous(breaks = scales::breaks_width(0.5))
  
if(building == T){
a
}else{
  ggsave("Paper/Figures/Workflow.png",a, units = "px")
}

## 3.1 Study sites

Field campaigns were conducted at three study sites across France and Spain. At each site, two locations were investigated (<a href="#fig-location_sites" class="quarto-xref">Figure 1</a>). The Aven & Belon estuaries in South Brittany, France (<a href="#fig-location_sites" class="quarto-xref">Figure 1</a> A & C) are dynamic ria-type systems hosting diverse habitats, including tidal flats and subtidal zones with coarse, marine-origin sediments ([Castaing and Guilcher, 1995](#ref-Castaing1995); [Michel et al., 2021](#ref-Michel2021)). These habitats support key benthic species such as *Scrobicularia plana*, *Cerastoderma edule*, and *Tellina tenuis*, which play essential roles in sediment bioturbation and nutrient cycling ([Blanchet et al., 2014](#ref-Blanchet2014); [Tankoua et al., 2011](#ref-Tankoua2011)). These estuaries serve as a nursery for juvenile fish and a feeding ground for migratory birds, with their ecological productivity driven by a mix of euryhaline and marine species adapted to salinity gradients ([Blanchet et al., 2014](#ref-Blanchet2014)). Oyster farming, particularly *Crassostrea gigas*, is a dominant activity, altering sediment dynamics and local biodiversity ([Michel et al., 2021](#ref-Michel2021)). Despite its ecological richness, the estuary faces pressures from nutrient loading and physical alteration ([Tankoua et al., 2011](#ref-Tankoua2011)).

In [ ]:
"The Ria d'Étel, located in Brittany, France, is a macrotidal estuary characterized by its unique hydrodynamics and biodiversity (@fig-location_sites B). Influenced predominantly by tidal regimes, the estuary exhibits high-energy zones with strong currents reaching up to 2.5 m/s, shaping both sediment deposition and ecological habitats [@Portas2023]. The estuary supports diverse benthic communities, with sedimentary organic matter originating from both terrestrial inputs and marine sources, contributing to nutrient cycling and benthic fluxes [@Jeanneau2023]. Vegetation gradients transition from halophytic plants in saline zones to freshwater species upstream, reflecting the estuary's salinity dynamics and ecological complexity [@Cianfaglione2018]. This estuary is also notable for its shellfish farming, with species like *Crassostrea gigas* cultivated extensively. The presence of filter-feeding organisms such as sponges (*Hymeniacidon perlevis*) enhances water quality by mitigating bacterial loads and promoting bioremediation [@Gentric2024]. However, the estuary faces environmental pressures, including nutrient enrichment from agricultural runoff and anthropogenic impacts on sedimentary processes.
"

The Saja-Besaya Estuary, situated along the Cantabrian Coast in northern Spain, is characterized by the confluence of the Saja and Besaya rivers near Torrelavega (<a href="#fig-location_sites" class="quarto-xref">Figure 1</a> C). The estuary, also known as San Martín de la Arena or Suances Estuary, has been subject to significant anthropogenic pressures, including industrial developments throughout the 20th century. These activities have led to contamination from mining, paper manufacturing, and carbonate discharges, classifying the estuary as highly polluted near its upper reaches ([Ortega et al., 2005](#ref-ortega2005fluxes)). This contamination impacted the water quality and biodiversity, with minimal aquatic life and sparse riverbank vegetation in its lower sections ([Romero et al., 2008](#ref-romero2008sintering)).

In [ ]:
######### Aven Belon 

mask_aven_belon <- read_sf("Data/shp/mask/mask_site_Aven_Belon_32630.shp")

HT <-  list.files("Data/Sentinel2/Aven_Belon/S2A_MSIL2A_20231107T111251_N0509_R137_T30TVT_20231107T144858.SAFE/GRANULE/L2A_T30TVT_A043749_20231107T111246/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_HT <- rast(HT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_aven_belon, mask = T) 
B8_HT <- B8_HT - 1000

B4_HT <- rast(HT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_aven_belon, mask = T)
B4_HT <- B4_HT -1000

NDVI_HT <- (B8_HT-B4_HT)/(B8_HT+B4_HT)


LT <-  list.files("Data/Sentinel2/Aven_Belon/S2A_MSIL2A_20240624T110641_N0510_R137_T30TVT_20240624T153247.SAFE/GRANULE/L2A_T30TVT_A047038_20240624T111001/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_LT <- rast(LT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_aven_belon, mask = T) 
B8_LT <- B8_LT - 1000

B4_LT <- rast(LT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_aven_belon, mask = T)
B4_LT <- B4_LT -1000

NDVI_LT <- (B8_LT-B4_LT)/(B8_LT+B4_LT)


intertidal_Aven_Belon <- NDVI_LT > 0.05 & NDVI_HT < -0.05


values(intertidal_Aven_Belon)[values(intertidal_Aven_Belon) == F] <- NA

mask_sf <- as.polygons(intertidal_Aven_Belon) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() %>% 
  smoothr::smooth(method = "ksmooth", smoothness = 3)
  
# plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_intertidal_Aven_Belon.shp")


Land_Aven_Belon <- NDVI_LT > 0.05 & NDVI_HT > 0.05


values(Land_Aven_Belon)[values(Land_Aven_Belon) == F] <- NA

mask_sf <- as.polygons(Land_Aven_Belon) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes()
  # smoothr::smooth(method = "ksmooth", smoothness = 3)
  
# plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_Land_Aven_Belon.shp")



######### ETEL

mask_Etel <- read_sf("Data/shp/mask/mask_site_Etel_32630.shp")

HT <-  list.files("Data/Sentinel2/Etel/S2A_MSIL2A_20231008T110941_N0509_R137_T30TVT_20231008T171550.SAFE/GRANULE/L2A_T30TVT_A043320_20231008T111641/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_HT <- rast(HT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_Etel, mask = T) 
B8_HT <- B8_HT - 1000

B4_HT <- rast(HT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_Etel, mask = T)
B4_HT <- B4_HT -1000

NDVI_HT <- (B8_HT-B4_HT)/(B8_HT+B4_HT)

LT <-  list.files("Data/Sentinel2/Etel/S2B_MSIL2A_20230718T112119_N0509_R037_T30TVT_20230718T124824.SAFE/GRANULE/L2A_T30TVT_A033239_20230718T112118/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_LT <- rast(LT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_Etel, mask = T) 
B8_LT <- B8_LT - 1000

B4_LT <- rast(LT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_Etel, mask = T)
B4_LT <- B4_LT -1000

NDVI_LT <- (B8_LT-B4_LT)/(B8_LT+B4_LT)


intertidal_Etel <- NDVI_LT > 0.05 & NDVI_HT < -0.05


values(intertidal_Etel)[values(intertidal_Etel) == F] <- NA

mask_sf <- as.polygons(intertidal_Etel) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() %>% 
  smoothr::smooth(method = "ksmooth", smoothness = 3)
  
# plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_intertidal_Etel.shp")


Land_Etel <- NDVI_LT > 0.05 & NDVI_HT > 0.05


values(Land_Etel)[values(Land_Etel) == F] <- NA

mask_sf <- as.polygons(Land_Etel) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() 
  
plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_Land_Etel.shp")

######### SAJA

mask_SAJA <- read_sf("Data/shp/mask/mask_site_Saja_32630.shp")


HT <-  list.files("Data/Sentinel2/Saja/S2A_MSIL2A_20181004T110911_N0500_R137_T30TVP_20230620T014212.SAFE/GRANULE/L2A_T30TVP_A017151_20181004T111207/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_HT <- rast(HT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_SAJA, mask = T) 
B8_HT <- B8_HT - 1000

B4_HT <- rast(HT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_SAJA, mask = T)
B4_HT <- B4_HT -1000

NDVI_HT <- (B8_HT-B4_HT)/(B8_HT+B4_HT)


LT <-  list.files("Data/Sentinel2/Saja/S2A_MSIL2A_20230928T110831_N0509_R137_T30TVP_20230928T171657.SAFE/GRANULE/L2A_T30TVP_A043177_20230928T111532/IMG_DATA/R10m/", pattern = ".jp2",recursive =T,full.names = T)

B8_LT <- rast(LT[5]) %>% 
  `names<-`("B08") %>% 
  crop(mask_SAJA, mask = T) 
B8_LT <- B8_LT - 1000

B4_LT <- rast(LT[4])%>% 
  `names<-`("B04") %>% 
  crop(mask_SAJA, mask = T)
B4_LT <- B4_LT -1000

NDVI_LT <- (B8_LT-B4_LT)/(B8_LT+B4_LT)


intertidal_Saja <- NDVI_LT > 0.05 & NDVI_HT < -0.05


values(intertidal_Saja)[values(intertidal_Saja) == F] <- NA

mask_sf <- as.polygons(intertidal_Saja) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() %>% 
  smoothr::smooth(method = "ksmooth", smoothness = 3)
  
# plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_intertidal_Saja.shp")


Land_Saja <- NDVI_LT > 0.05 & NDVI_HT > 0.05

values(Land_Saja)[values(Land_Saja) == F] <- NA

mask_sf <- as.polygons(Land_Saja) %>% 
  st_as_sf() %>% 
  st_cast("POLYGON") %>% 
  mutate(area = st_area(geometry)) %>% 
  dplyr::filter(as.numeric(area) > 6500) %>% 
  nngeo::st_remove_holes() %>% 
  smoothr::smooth(method = "ksmooth", smoothness = 3)
  
plot(mask_sf)

write_sf(mask_sf,"Data/shp/Intertidal_sites/mask_Land_Saja.shp")

In [ ]:
sovereignty10 <- ne_countries(scale = 10, returnclass = "sf")

world_map <- sovereignty10 %>% 
  st_as_sf() %>% 
  dplyr::filter(sovereignt%in%c("Spain","France","Portugal",
                                "Italy","Andorra",
                                "United Kingdom",
                                "Switzerland","Belgium",
                                "Germany","Luxembourg") ) 

bbox_europe <- st_bbox(c(xmin = -20, ymin = 34,
                         xmax = 20, ymax = 55) ,
                       crs = st_crs(world_map) ) 

world_map<-st_make_valid(world_map) 

european_union_map_cropped <- st_crop(world_map, bbox_europe)  %>% 
  st_transform("+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs ")  


UnitedKingdom<-sovereignty10 %>% 
  st_as_sf() %>% 
  dplyr::filter(sovereignt%in%c("United Kingdom") ) %>% 
  st_cast("POLYGON") 

bbox_UK <- st_bbox(c(xmin = -20, ymin = 45,
                         xmax = 20, ymax = 55) ,
                       crs = st_crs(UnitedKingdom) ) 

UnitedKingdom<-st_make_valid(UnitedKingdom) 

UK_map_cropped <- st_crop(UnitedKingdom, bbox_UK)  %>% 
  st_transform("+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs ") 

Europe_sf<-european_union_map_cropped %>% 
  dplyr::bind_rows(UK_map_cropped) 


Projects<-data.frame(
  Name=c(
  "France - Aven",
  "France - Belon",
  "Spain - Saja Estuary"
  ) ,
  Long=c(-3.187818,
         -3.187818, 
         -4.025402) ,
  Lat=c(47.697566, 
        47.697566,
        43.408356) 
  )  %>% 
  st_as_sf(coords=c("Long","Lat") )  %>% 
  st_set_crs("EPSG:4326")  %>% 
  st_transform("+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs ")  

Projects_df<-Projects %>%
  dplyr::mutate(lon = sf::st_coordinates(.) [,1],
                lat = sf::st_coordinates(.) [,2]) %>% 
  sf::st_set_geometry(NULL)

Miniworld_map <- sovereignty10 %>% 
  st_as_sf()

sf_use_s2(FALSE)

bbox_EU <- st_bbox(c(xmin = -30, ymin = 20,
                         xmax = 50, ymax = 70) ,
                       crs = st_crs(Miniworld_map) ) 

  MiniEU_map<-st_crop(Miniworld_map, bbox_EU)  %>% 
  st_transform("+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs ")  

  df_P1<-Projects_df %>% 
    mutate(ID = c(1:nrow(.))) %>% 
    mutate(Site = case_when(ID == 1 ~ "Aven Estuary",
                            ID == 2 ~ "Bélon Estuary",
                            ID == 3 ~ "Saja Estuary",
                            TRUE ~ "NA"),
           ID = case_when(ID == 1 ~ "A",
                          ID == 2 ~ "B",
                          TRUE ~ "C"),
           lon_label = 2900000,
           lat_label = case_when(Site == "Aven Estuary" ~ lat  - 200000,
                                 Site == "Bélon Estuary" ~ lat + 50000,
                                 Site == "Saja Estuary" ~ lat  - 250000,),
           lat_ID = lat_label + 100000) %>% 
    dplyr::select(c(lon,lat,lon_label,lat_label,lat_ID,ID,Site))
    
    scaleFUN <- function(x) paste0(sprintf("%.2f", x),"°N")
(
p1 <- ggplot(MiniEU_map) +
  geom_sf(linewidth = 0.5, alpha = 0.93, fill = "#CFCFCF", colour = "grey30") +
  geom_segment(data = df_P1, aes(x = lon_label, xend = lon, yend = lat, y =lat_label, group = ID), color = "goldenrod4",size = 1)+
  geom_point(data = df_P1, aes(x = lon, y =lat), color = "darkred", alpha = 0.8, size = 5)+
  geom_label(data = df_P1, aes(x = lon_label, y =lat_label, label = Site), fill = "#FFD27D",size = 5)+
  geom_label(data = df_P1, aes(x = lon_label, y = lat_ID, label = c("B","A","C")), size = 6)+
  coord_sf(
    xlim = c(2600000, 4100000),
    ylim = c(1600000, 3100000), 
    expand = FALSE
  ) +
  theme_Bede_Map() +
  labs(x = "Longitude", y = "Latitude") +
  scale_x_continuous(breaks = 0) +
  scale_y_continuous(breaks = 42, labels = scaleFUN) +
  # theme_void() +
  theme(
    plot.margin = unit(c(0, 0, 0, 0), "cm"),
    axis.title = element_blank(),
    axis.ticks = element_blank(),
    axis.text.x = element_text(size = 20, vjust = 7.1, hjust = 0.8),
    axis.text.y = element_text(size = 20, vjust = -5, hjust = 0, angle = 90),
    # panel.border = element_rect(color = "black", fill = NA),
    # panel.background = element_rect(fill = "white"),
    # plot.background = element_rect(fill = "white", color = NA),
    # legend.background = element_rect(fill = "transparent"),
    # panel.grid.major = element_line(color = "grey", linetype = "dashed", linewidth = 0.5),
    # panel.grid.minor = element_line(color = "grey", linetype = "dotted", linewidth = 0.25)
  ))



ggsave("Paper/Figures/High_res/Figure1/Map_Drone_Sites.png",p1,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_Drone_Sites.png",p1,width= 10, height=10, dpi = 200) 

In [ ]:
create_square_sf <- function(side_length, center_lonlat) {
  # Validate input
  if (length(center_lonlat) != 2) {
    stop("Please provide the center coordinates as a vector: c(longitude, latitude).")
  }
  if (!is.numeric(side_length) || side_length <= 0) {
    stop("Please provide a positive numeric value for the side length.")
  }
  
  # Define the center point in lon/lat
  center_point <- st_sfc(st_point(center_lonlat), crs = 4326)
  
  # Transform to EPSG:32630
  center_point_utm <- st_transform(center_point, 32630)
  center_coords <- st_coordinates(center_point_utm)
  
  # Calculate half side length in meters
  half_side <- side_length / 2
  
  # Create the square corners in UTM
  square_coords <- matrix(c(
    center_coords[1] - half_side, center_coords[2] - half_side,
    center_coords[1] + half_side, center_coords[2] - half_side,
    center_coords[1] + half_side, center_coords[2] + half_side,
    center_coords[1] - half_side, center_coords[2] + half_side,
    center_coords[1] - half_side, center_coords[2] - half_side # Close the square
  ), ncol = 2, byrow = TRUE)
  
  # Create a polygon from the coordinates
  square_polygon <- st_polygon(list(square_coords))
  
  # Create an sf object
  square_sf <- st_sfc(square_polygon, crs = 32630)
  
  return(square_sf)
}

In [ ]:
It_Saja <- "Data/shp/Intertidal_sites/mask_intertidal_Saja.shp" %>% 
  read_sf()

Land_Saja <- "Data/shp/Intertidal_sites/mask_Land_Saja.shp" %>% 
  read_sf()

ext_plot_Saja <- st_bbox(create_square_sf(5000, c(-4.025987, 43.4200065)))



Flight_Saja<-data.frame(
  Name=c(
  "Marisma de \nCortiguera",
  "Marisma de \nCudón"),
  Long=c(-4.029591,
         -4.031800) ,
  Lat=c(43.409738,
        43.414434)
  )  %>%
  st_as_sf(coords=c("Long","Lat") )  %>%
  st_set_crs("EPSG:4326")  %>%
  st_transform(crs(Land_Saja))  %>%
  dplyr::mutate(lon = sf::st_coordinates(.) [,1],
                lat = sf::st_coordinates(.) [,2]) %>%
  sf::st_set_geometry(NULL) %>% 
  mutate(lon_label = case_when(Name == "Marisma de \nCudón" ~ 418639.3,
                               Name == "Marisma de \nCortiguera" ~ lon + 2000),
         lat_label = case_when(Name == "Marisma de \nCudón" ~ lat + 1000,
                               Name == "Marisma de \nCortiguera" ~ lat + 500))

(plot_Saja <- ggplot() +
  geom_sf(data = Land_Saja, linewidth = 0.05, alpha = 0.93, fill = "grey80") +
  geom_sf(data = It_Saja, linewidth = 0.05, alpha = 0.93, fill = "goldenrod")+
    geom_label(aes(x = as.numeric(ext_plot_Saja[1]) + (as.numeric(ext_plot_Saja[3])-as.numeric(ext_plot_Saja[1]))*0.1 , 
                   y = as.numeric(ext_plot_Saja[2]) + (as.numeric(ext_plot_Saja[4])-as.numeric(ext_plot_Saja[2]))*0.1
                   , label = "C"), 
               size = 10,
               alpha = 0.5 )+
  coord_sf(
    xlim = c(ext_plot_Saja[1], ext_plot_Saja[3]),
    ylim = c(ext_plot_Saja[2], ext_plot_Saja[4]),
    expand = F
  ) +
  geom_segment(data = Flight_Saja, aes(x = lon_label, xend = lon, yend = lat, y =lat_label, group = Name), color = "goldenrod4",size = 1)+
  geom_point(data = Flight_Saja, aes(x = lon, y =lat), color = "darkred", alpha = 0.8, size = 5)+
  geom_label(data = Flight_Saja, aes(x = lon_label, y =lat_label, label = Name), fill = "#FFD27D",size = 5)+
  theme_Bede_Map()+
  labs(x="Longitude",
       y="Latitude")+
  scale_x_continuous(breaks = -4.01) +
  scale_y_continuous(breaks = 43.415) +
  # Move y-axis to the right
  theme(
    legend.background = element_rect(fill = alpha("white", 0)),
    plot.background = element_rect(fill = "white", color = NA),
    axis.text.x = element_text(vjust = 4.5,  size = 20),
    axis.text.y = element_text(angle = 90, vjust = -4, hjust = 0.5, size = 20),
    axis.title = element_blank(),
    axis.ticks = element_blank()
    )  # Adjust position if needed  )
)

ggsave("Paper/Figures/High_res/Figure1/Map_Saja.png",plot_Saja,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_Saja.png",plot_Saja,width= 10, height=10, dpi = 200)  

In [ ]:
It_AB <- "Data/shp/Intertidal_sites/mask_intertidal_Aven_Belon.shp" %>% 
  read_sf()

Land_AB <- "Data/shp/Intertidal_sites/mask_Land_Aven_Belon.shp" %>% 
  read_sf()

ext_plot_AB <- st_bbox(create_square_sf(8000, c(-3.780146, 47.810000)))


Flight_AB<-data.frame(
  Name=c(
  "Notre-Dame De Tremor"),
  Long=c(-3.748634) ,
  Lat=c(47.837923)
  )  %>%
  st_as_sf(coords=c("Long","Lat") )  %>%
  st_set_crs("EPSG:4326")  %>%
  st_transform(crs(Land_Saja))  %>%
  dplyr::mutate(lon = sf::st_coordinates(.) [,1],
                lat = sf::st_coordinates(.) [,2]) %>%
  sf::st_set_geometry(NULL) %>% 
  mutate(lon_label = case_when(Name == "Notre-Dame De Tremor" ~  as.numeric(ext_plot_AB[1]) + (as.numeric(ext_plot_AB[4])-as.numeric(ext_plot_AB[2]))*0.35),
         lat_label = case_when(Name == "Notre-Dame De Tremor" ~  as.numeric(ext_plot_AB[2]) + (as.numeric(ext_plot_AB[4])-as.numeric(ext_plot_AB[2]))*0.85))

(Plot_Aven <- ggplot() +
  geom_sf(data = Land_AB, linewidth = 0.05, alpha = 0.93, fill = "grey80") +
  geom_sf(data = It_AB, linewidth = 0.05, alpha = 0.93, fill = "goldenrod") +
    geom_label(aes(x = as.numeric(ext_plot_AB[1]) + (as.numeric(ext_plot_AB[3])-as.numeric(ext_plot_AB[1]))*0.1 , 
                   y = as.numeric(ext_plot_AB[2]) + (as.numeric(ext_plot_AB[4])-as.numeric(ext_plot_AB[2]))*0.1
                   , label = "A"), 
               size = 10,
               alpha = 0.5 )+
  coord_sf(
    xlim = c(ext_plot_AB[1], ext_plot_AB[3]),
    ylim = c(ext_plot_AB[2], ext_plot_AB[4]),
    expand = F
  ) +
  geom_segment(data = Flight_AB, aes(x = lon_label, xend = lon, yend = lat, y =lat_label, group = Name), color = "goldenrod4",size = 1)+
  geom_point(data = Flight_AB, aes(x = lon, y =lat), color = "darkred", alpha = 0.8, size = 5)+
  geom_label(data = Flight_AB, aes(x = lon_label, y =lat_label, label = Name), fill = "#FFD27D",size = 5)+
  theme_Bede_Map()+
  labs(x="Longitude",
       y="Latitude")+
  scale_x_continuous(breaks = -3.725) +
  scale_y_continuous(breaks = 47.82) +
  # Move y-axis to the right
  theme(
    legend.background = element_rect(fill = alpha("white", 0)),
    plot.background = element_rect(fill = "white", color = NA),
    axis.text.x = element_text(vjust = 7.5,  size = 20),
    axis.text.y = element_text(angle = 90, vjust = -7, hjust = 0.5, size = 20),
    axis.title = element_blank(),
    axis.ticks = element_blank(),
    legend.box = "vertical",  # Stack legends vertically
    legend.box.just = "left",  # Align the legends to the left
    legend.spacing.y = unit(0.0, 'mm'),  # Adjust the spacing between items
    legend.key = element_rect(fill = "white", color = "black"),  # Ensure clear legend keys
    legend.box.margin = margin(5, 5, 5, 5),
    panel.grid.major = ggplot2::element_line(linetype = "dotted", 
      colour = alpha("grey30",0.5), linewidth = 0.25)
    )  # Adjust position if needed  )
)

ggsave("Paper/Figures/High_res/Figure1/Map_AB.png",Plot_Aven,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_AB.png",Plot_Aven,width= 10, height=10, dpi = 200)  

In [ ]:
It_AB <- "Data/shp/Intertidal_sites/mask_intertidal_Aven_Belon.shp" %>% 
  read_sf()

Land_AB <- "Data/shp/Intertidal_sites/mask_Land_Aven_Belon.shp" %>% 
  read_sf()

ext_plot_B <- st_bbox(create_square_sf(8000, c(-3.690000, 47.800000)))


Flight_AB<-data.frame(
  Name=c("Pont du Guilly"),
  Long=c(-3.655220) ,
  Lat=c(47.822408)
  )  %>%
  st_as_sf(coords=c("Long","Lat") )  %>%
  st_set_crs("EPSG:4326")  %>%
  st_transform(crs(Land_Saja))  %>%
  dplyr::mutate(lon = sf::st_coordinates(.) [,1],
                lat = sf::st_coordinates(.) [,2]) %>%
  sf::st_set_geometry(NULL) %>% 
  mutate(lon_label = case_when(Name == "Pont du Guilly" ~ as.numeric(ext_plot_B[1]) + (as.numeric(ext_plot_B[3])-as.numeric(ext_plot_B[1]))*0.7),
         lat_label = case_when(Name ==  "Pont du Guilly" ~  as.numeric(ext_plot_B[2]) + (as.numeric(ext_plot_B[4])-as.numeric(ext_plot_B[2]))*0.7))

(plot_Belon <- ggplot() +
  geom_sf(data = Land_AB, linewidth = 0.05, alpha = 0.93, fill = "grey80") +
  geom_sf(data = It_AB, linewidth = 0.05, alpha = 0.93, fill = "goldenrod") +
    geom_label(aes(x = as.numeric(ext_plot_B[1]) + (as.numeric(ext_plot_B[3])-as.numeric(ext_plot_B[1]))*0.1 , 
                   y = as.numeric(ext_plot_B[2]) + (as.numeric(ext_plot_B[4])-as.numeric(ext_plot_B[2]))*0.1
                   , label = "B"), 
               size = 10,
               alpha = 0.5 )+
  coord_sf(
    xlim = c(ext_plot_B[1], ext_plot_B[3]),
    ylim = c(ext_plot_B[2], ext_plot_B[4]),
    expand = F
  ) +
  geom_segment(data = Flight_AB, aes(x = lon_label, xend = lon, yend = lat, y =lat_label, group = Name), color = "goldenrod4",size = 1)+
  geom_point(data = Flight_AB, aes(x = lon, y =lat), color = "darkred", alpha = 0.8, size = 5)+
  geom_label(data = Flight_AB, aes(x = lon_label, y =lat_label, label = Name), fill = "#FFD27D",size = 5)+
  theme_Bede_Map()+
  labs(x="Longitude",
       y="Latitude")+
  scale_x_continuous(breaks = -3.67) +
  scale_y_continuous(breaks = 47.82) +
  # Move y-axis to the right
  theme(
    legend.background = element_rect(fill = alpha("white", 0)),
    plot.background = element_rect(fill = "white", color = NA),
    axis.text.x = element_text(vjust = 7.5,  size = 20),
    axis.text.y = element_text(angle = 90, vjust = -7, hjust = 0.5, size = 20),
    axis.title = element_blank(),
    axis.ticks = element_blank(),
    legend.box = "vertical",  # Stack legends vertically
    legend.box.just = "left",  # Align the legends to the left
    legend.spacing.y = unit(0.0, 'mm'),  # Adjust the spacing between items
    legend.key = element_rect(fill = "white", color = "black"),  # Ensure clear legend keys
    legend.box.margin = margin(5, 5, 5, 5),
    panel.grid.major = ggplot2::element_line(linetype = "dotted", 
      colour = alpha("grey30",0.5), linewidth = 0.25)
    )  # Adjust position if needed  )
)

ggsave("Paper/Figures/High_res/Figure1/Map_AB.png",plot_AB,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_AB.png",plot_AB,width= 10, height=10, dpi = 200)  

In [ ]:
It_Etel <- "Data/shp/Intertidal_sites/mask_intertidal_Etel.shp" %>% 
  read_sf()

Land_Etel <- "Data/shp/Intertidal_sites/mask_Land_Etel.shp" %>% 
  read_sf()



ext_plot_Etel <- st_bbox(create_square_sf(10000, c(-3.186821, 47.683477)))


Flight_Etel<-data.frame(
  Name=c(
  "Berringue",
  "Lorois"),
  Long=c(-3.194274,
         -3.205714) ,
  Lat=c(47.698978,
        47.675159)
  )  %>%
  st_as_sf(coords=c("Long","Lat") )  %>%
  st_set_crs("EPSG:4326")  %>%
  st_transform(crs(Land_Saja))  %>%
  dplyr::mutate(lon = sf::st_coordinates(.) [,1],
                lat = sf::st_coordinates(.) [,2]) %>%
  sf::st_set_geometry(NULL) %>% 
  mutate(lon_label = case_when(Name == "Berringue" ~ lon - 1000,
                               Name == "Lorois" ~ lon + 2000),
         lat_label = case_when(Name ==  "Berringue" ~ lat + 1000,
                               Name == "Lorois" ~ lat - 1000))


(plot_Etel <- ggplot() +
  geom_sf(data = Land_Etel, linewidth = 0.05, alpha = 0.93, fill = "grey80") +
  geom_sf(data = It_Etel, linewidth = 0.05, alpha = 0.93, fill = "goldenrod") +
    geom_label(aes(x = as.numeric(ext_plot_Etel[1]) + (as.numeric(ext_plot_Etel[3])-as.numeric(ext_plot_Etel[1]))*0.1 , 
                   y = as.numeric(ext_plot_Etel[2]) + (as.numeric(ext_plot_Etel[4])-as.numeric(ext_plot_Etel[2]))*0.1
                   , label = "B"), 
               size = 10,
               alpha = 0.5 )+
  coord_sf(
    xlim = c(ext_plot_Etel[1], ext_plot_Etel[3]),
    ylim = c(ext_plot_Etel[2], ext_plot_Etel[4]),
    # ndiscr = 1,
    expand = FALSE
  ) +
  scale_x_continuous(breaks = -3.2) +
  scale_y_continuous(breaks = 47.68) +
  geom_segment(data = Flight_Etel, aes(x = lon_label, xend = lon, yend = lat, y =lat_label, group = Name), color = "goldenrod4",size = 1)+
  geom_point(data = Flight_Etel, aes(x = lon, y =lat), color = "darkred", alpha = 0.8, size = 5)+
  geom_label(data = Flight_Etel, aes(x = lon_label, y =lat_label, label = Name), fill = "#FFD27D",size = 5)+
  theme_Bede_Map() +
  labs(x = "Longitude", y = "Latitude")+
  # scale_x_continuous(breaks = seq(floor(ext_plot_Etel[1]), ceiling(ext_plot_Etel[3]), by = 500)) +
  # scale_y_continuous(breaks = seq(floor(ext_plot_Etel[2]), ceiling(ext_plot_Etel[4]), by = 500)) +

  theme(
    legend.background = element_rect(fill = alpha("white", 0)),
    plot.background = element_rect(fill = "white", color = NA),
    axis.text.x = element_text(vjust = 7.5,  size = 20),
    axis.text.y = element_text(angle = 90, vjust = -7, hjust = 0.5, size = 20),
    axis.title = element_blank(),
    axis.ticks = element_blank(),
    legend.box = "vertical",
    legend.box.just = "left",
    legend.spacing.y = unit(0.0, 'mm'),
    legend.key = element_rect(fill = "white", color = "black"),
    legend.box.margin = margin(5, 5, 5, 5),
    panel.grid.major = element_line(
      linetype = "dotted",
      colour = alpha("grey30", 0.5),
      linewidth = 0.25
    )
  )
)

ggsave("Paper/Figures/High_res/Figure1/Map_Etel.png",plot_AB,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_Etel.png",plot_AB,width= 10, height=10, dpi = 200)  

In [ ]:
(final_map <- ((p1 | Plot_Aven) / (plot_Belon | plot_Saja))  + 
  plot_layout(guides = "collect")& 
    # theme_void()&
  theme(legend.position = "bottom",
        plot.margin = unit(c(0, 0, 0, 0), "cm"),
    axis.title = element_blank(),
    axis.ticks = element_blank(),
    axis.text.x = element_text(size = 20, vjust = 6, hjust = 0.8),
    axis.text.y = element_text(size = 20, vjust = -5, hjust = 0, angle = 90))
)

ggsave("Paper/Figures/High_res/Figure1/Map_site.png",final_map,width= 10, height=10, dpi = 400)  
ggsave("Paper/Figures/Low_res/Figure1/Map_site.png",final_map,width= 10, height=10, dpi = 200)

In [ ]:
knitr::include_graphics("Figures/Low_res/Figure1/Map_site.png")

## 3.2 Remote sensing data acquisition and pre-processing

In [ ]:
knitr::include_graphics("./Figures/Low_res/FigurePictures_censored.png")

### 3.2.1 Hyperspectral measurements

At each location, hyperspectral reflectance signatures were recorded using an ASD FieldSpec HandHeld 2 spectroradiometer (Malvern Panalytical, Worcestershire, UK), which measures reflectance from 325 to 1075 nm with a spectral resolution of approximately 1 nm (<a href="#fig-PictureFigure" class="quarto-xref">Figure 2</a> D). Each spectrum was subsequently smoothed using a Savitzky–Golay filter ([Savitzky and Golay, 1964](#ref-savitzky1964smoothing)) with a third-order polynomial and an 11-point window, selected to minimize noise while preserving salient spectral features. After this initial smoothing, the first and second derivatives were computed using a central difference approximation (<a href="#eq-SecondDerivative" class="quarto-xref">Equation 1</a>).

<span id="eq-SecondDerivative">$$
f''(\lambda_i) \approx \frac{f(\lambda_{i+1}) - 2f(\lambda_i) + f(\lambda_{i-1})}{(\Delta \lambda)^2}
 \qquad(1)$$</span>

where $f(\lambda_i)$ is the reflectance at wavelength $\lambda_i$ and $\Delta \lambda$ is the uniform spectral sampling interval.

### 3.2.2 Drone data

A total of four drone flights were conducted across the three study sites. All flights were performed at an altitude of 120 m and a speed of 10 m·s⁻¹. Two flights were carried out in the Saja Estuary on June 25, 2024, covering areas of 20.4 hectares (Marisma de Cortiguera) and 8.4 hectares (Marisma de Cudón), respectively (<a href="#fig-location_sites" class="quarto-xref">Figure 1</a>). The other two flights took place in the Belon and Aven estuaries on April 11, 2024, covering areas of 21.3 hectares and 26.7 hectares, respectively.

#### 3.2.2.1 Multispectral data

At each location, reflectance images with of 1.2 million pixels were captured using a DJI Matrice 300 quadcopter drone equipped with a Micasense RedEdge Dual MX multispectral camera. The camera recorded data across ten spectral bands, spanning from blue to near-infrared (NIR) wavelengths (444, 475, 531, 560, 650, 668, 705, 717, 740, and 840 nm). To ensure consistent lighting conditions, the drone’s flight trajectory was aligned to maintain a solar azimuth angle of 90 degrees. Image acquisition was carried out with an overlap of 70% between side-by-side images and 80% between successive images along the flight path. A downwelling light sensor (DLS2) was used to measure real-time irradiance, enabling the correction of reflectance values for variations in light intensity caused by changing cloud cover during the flight. The raw image data were subsequently calibrated to reflectance using a calibration panel with ~50% reflectivity, provided by the camera’s manufacturer. Images were processed using structure-from-motion photogrammetry software ([Agisoft, 2019](#ref-agisoft)) to generate multispectral ortho-mosaics for each flight. The ortho-mosaicking workflow was consistent across all flights. Initially, key tie points were identified within each image and across overlapping images to create a sparse point cloud. This point cloud was refined by removing noisy points using a reprojection accuracy metric. Subsequently, a dense point cloud was generated using a structure-from-motion algorithm. A digital surface model (DSM) was then created through surface interpolation of the dense point cloud, which served as the basis for reconstructing the multispectral ortho-image ([Nebel et al., 2020](#ref-nebel2020review)). The resolution of the multispectral ortho-mosaic obtained was 8 cm per pixel.

#### 3.2.2.2 LiDAR data

Using the Matrice 300 Series Dual Gimbal Connector, a DJI Zenmuse L1 LiDAR and RGB sensor was mounted on the drone alongside the multispectral camera. This setup enabled the simultaneous capture of LiDAR point clouds, high-resolution RGB images, and multispectral images collected by the MicaSense RedEdge Dual MX during the same flight. The same processing workflow as <a href="#sec-photo" class="quarto-xref">Section 3.2.2.1</a> was applied to process LiDAR RGB images, resulting in ortho-mosaic with a resolution of 2.5 cm per pixel. Since the mapping focused solely on surfaces without dense vegetation, the LiDAR measured only a single return. Operating in repetitive scanning mode with a sampling rate of 240 kHz, the system achieved a point density of 350 points per square meter. The LiDAR point cloud was extracted and converted into LAS format using DJI Terra software. The LAS point cloud was then imported into Agisoft Metashape ([Agisoft, 2019](#ref-agisoft)) to generate a Digital Surface Model (DSM) with a resolution of 2.5 cm. From the DSM, the slope of each pixel based on a grid of 8 surrounding pixels was computed using the terrain function of the ‘terra’ package in R ([Hijmans, 2024](#ref-terrapck)). The angle of the mudflat was categorized into three classes: Flat (angle \< 10°), Angled (10° ≤ angle ≤ 40°), and Steep (angle \> 40°).

## 3.3 Scene classification

In [ ]:
library(tidyverse)
library(flextable)

df_training <- "../Data/Training_DISCOV/DISCOV_Invasea_Saja_filtered.txt" %>% 
  read.delim(sep = ";")


brdr1 <- fp_border_default(color = "black", width = 0.5)
brdr2 <- fp_border_default(color = "grey", width = 1)
brdr3 <- fp_border_default(color = "grey40", width = 1)

i = 16.5 # width of the side borders in the word_document output (in centimeters)
w = i*0.3937 # width of the side borders in the word_document output (in inches)
 
df_table <- df_training %>% 
  dplyr::filter(True_Class != "SunGlint") %>% 
  mutate(DISCOV1_n = case_when(True_Class == "Bacillariophyceae" ~ 4475,
                          True_Class == "Chlorophyta" ~ 17140,
                          True_Class == "Magnoliopsida" ~ 221065,
                          True_Class == "Phaeophyta" ~ 169936,
                          True_Class == "Rhodophyta" ~ 5771,
                          True_Class == "Water" ~ 83677,
                          True_Class == "Sediment" ~ 95126),
         Name = case_when(True_Class == "Bacillariophyceae" ~ "Benthic Diatoms",
                          True_Class == "Chlorophyta" ~ "Green macroalgae",
                          True_Class == "Magnoliopsida" ~ "Seagrass",
                          True_Class == "Phaeophyta" ~ "Brown macroalgae",
                          True_Class == "Rhodophyta" ~ "Red macroalgae",
                          T ~ True_Class),
         True_Class = case_when(True_Class == "Sediment" ~ "-",
                                True_Class == "Water" ~ "-",
                                T ~ True_Class),
         `Difference with DISCOV v1.0` = round((n)/DISCOV1_n,2),
         `Difference with DISCOV v1.0` = case_when(`Difference with DISCOV v1.0` == 1 ~ "-",
                                                   T ~ paste0("x",`Difference with DISCOV v1.0`))) 

In a previous study we developed a neural network classification model (DISCOV; Oiry et al. ([2024](#ref-rs16234383))), previously applied with success to Micasense reflectance data for mapping intertidal vegetation along the Portuguese and French Atlantic coasts, has been used in this study. The training dataset of DISCOV v1.0 was updated <a href="#fig-Workflow" class="quarto-xref">Figure 3</a>. As shown by Oiry et al. ([2024](#ref-rs16234383)) the DISCOV v1.0 model was trained using only 5771 Rhodophyceae pixel (3% of the training dataset). To fill this, gap the original training dataset of DISCOV v1.0 was updated using new training pixel coming from the 5 drone flights (<a href="#sec-DroneFlights" class="quarto-xref">Section 3.2</a>). A total of 427000 pixels were added to the DISCOV training dataset from version 1 (<a href="#sec-AnnexeA" class="quarto-xref">Section 7.1</a>).

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Linking to GEOS 3.11.2, GDAL 3.8.2, PROJ 9.3.1; sf_use_s2() is TRUE

To validate the DISCOV model, a user-friendly Shiny app was developed. This app enabled independent users to photo-interpret snapshots of the ortho-mosaic from each drone flight ([Chang et al., 2024](#ref-shinypck); [Simon, 2024](#ref-Simon2024ShinyApp)). Users could click on various parts of the snapshots to indicate the type of vegetation they believed was present. Using this method, 3 independent users contributed to creating a validation dataset of 6755 pixels across 79 snapshots distributed among the four drone flights (<a href="#sec-AnnexeB" class="quarto-xref">Section 7.2</a>). The validation dataset was then simplified into two classes: The presence or absence of Red Algae (<a href="#fig-Workflow" class="quarto-xref">Figure 3</a>).

In [ ]:
knitr::include_graphics("./Figures/Low_res/Flowchart gracillaria.png")

## 3.4 Historical Presence of *Gracilaria vermiculophylla* in the Bélon estuary

To assess the historical presence of *G. vermiculophylla* in the Belon Estuary, aerial imagery from flight campaigns was obtained via the IGN platform “Remonter Le Temps” ([IGN, 2024](#ref-RemonterLeTempsIGN)). Nine images were selected between 1952 and 2012 from the IGN platform and an additional one has been added for the year 2024 (<a href="#sec-AnnexeC" class="quarto-xref">Section 7.3</a>). Since most of the images retrieved from “Remonter Le Temps” were digitized versions of physical photographs, georeferencing was required.

For each date, polygons have been drawn around *G. vermiculophylla* patches by visual photo-interpretation. These polygons were used to calculate the total area of the mudflat covered by macroalgae within a common extent of 30 hectares in Pont de Guilly, located in the Belon Estuary, South Brittany, France.

## 3.5 Statistical analysis

We used a Generalized Linear Mixed Model (GLMM) within a Bayesian framework using the ‘brms’ package in R ([Bürkner, 2021](#ref-brm3), [2018](#ref-brm2), [2017](#ref-brm1)). The response variable, the cover of *G. vermiculophylla*, was modeled using a Beta distribution as a function of bathymetry elevation and the slope of the mudflat (categorized as Flat, Angled, Steep). A random intercept for site was included to account for potential hierarchical variation among sampling sites. The Beta distribution was chosen because the response variable is continuous and constrained between 0 and 1. We visually assessed sample vs. fitted residuals and quartile–quartile (Q-Q) plots to ensure that the model assumptions, including appropriate model fit and absence of patterns in residuals, were satisfied.

# 4. Results

## 4.1 Historical records in the Belon estuary

In [ ]:
library(tidyverse)
library(sf)
library(terra)
library(tidyterra)

img_list <- list.files("Data/RemonterLeTemps/img", full.names = T, include.dirs = F) %>% 
  as_tibble() %>% 
  rename(path ="value") %>% 
  dplyr::filter(!str_detect(path, "RAW")) %>% 
  mutate(filename = gsub(".*/","",path),
         source = case_when(str_detect(filename, "IGNF") ~ "IGN",
                          T ~ "Other"),
         year = as.integer(case_when(source == "IGN" ~ substr(filename,15,18),
                          T ~ substr(filename,1,4))))

mask <- "Data/RemonterLeTemps/shp/mask/IntertidalMask_Belon_RemonterLeTemps.shp" %>% 
  read_sf()

Pres_algae <- "Data/RemonterLeTemps/shp/shape_red_algae/Presence_Absence_RedAlgae_Date.shp" %>% read_sf()



Text_position <- data.frame(x = rep(ext(mask)[1]+0.9*(ext(mask)[2]-ext(mask)[1]),nrow(img_list)),
                            y = rep(ext(mask)[3]+0.9*(ext(mask)[4]-ext(mask)[3]),nrow(img_list)),
                            year = img_list$year)

for(i in 1:nrow(img_list)){
  

  img <- rast(img_list$path[i]) %>% 
    project(crs(mask)) %>% 
    crop(mask, mask = T)
  
  date_i <- Text_position[i,]
  # shp_i <- Pres_algae %>% 
  #   dplyr::filter(date == img_list$year[i])
  # 
  # if(img_list$year[i] == 2024){
  #   shp_i <- shp_i %>% 
  #     st_buffer(dist = 0.3) %>% 
  #     st_union()
  # }
  # 
  
Plot <-  
  if(nlyr(img) != 1){
    RGB(img)<-1:3
    
    ggplot()+
      geom_spatraster_rgb(data = img)
  }else{
    names(img) = "layer"
    ggplot()+
      geom_spatraster(img, mapping = aes(fill = layer), show.legend = F)+
      scale_fill_gradient(low = "black", high = "white", limits = c(0,240),na.value = NA, name = "Greyscale")
  }

a <- Plot +
    geom_text(data = date_i, aes(x = x, y = y, label = year),
              size = 7)+
  # geom_sf(data = shp_i, fill = NA, color = "darkred")+ 
   coord_sf(xlim = c(ext(mask)[1],ext(mask)[2]),
            ylim = c(ext(mask)[3],ext(mask)[4]),
            expand = F)+
    # geom_text(data = date_i, aes(x = x, y = y, label = year),
    #           size = 10)+
    theme_Bede()+
    theme(axis.text.x = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.x = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_blank(),
          axis.title.y = element_blank(),
          strip.clip = "inherit",
          panel.spacing = unit(0,"pt"), 
          plot.margin = unit(c(0, 0, 0, 0), "lines"))+
    if(img_list$year[i] == 2024){
    ggspatial::annotation_scale(location = "bl",
                                pad_y = unit(0.3, "cm"),
                                text_cex=1)
    }
  
  
a 
  
assign(paste0("plot_",as.character(img_list$year[i])),a)
  
}


# plot_all <- (plot_1952 + plot_1958)/
# (plot_1976 + plot_1978)/
# (plot_1982 + plot_1992)/
# (plot_1997 + plot_2012)/
# (plot_2019 + plot_2024)

plot_all <- (plot_1952 + plot_1958 + plot_1976)/
(plot_1978 + plot_1982 + plot_1992)/
(plot_1997 + plot_2012 + plot_2024)


ggsave("Paper/Figures/High_res/Historical_maps.png",plot_all, width = 10, height = 4.52, dpi = 400)
ggsave("Paper/Figures/Low_res/Historical_maps.png",plot_all, width = 10, height = 4.52, dpi = 200)

In [ ]:
library(tidyverse)
library(sf)
library(Utilities.Package)

mask <- "Data/RemonterLeTemps/shp/mask/IntertidalMask_Belon_RemonterLeTemps.shp" %>% 
  read_sf() %>% 
  mutate(area = st_area(.)) %>% 
  pull(as.numeric(area)[1])

Pres_algae <- "Data/RemonterLeTemps/shp/shape_red_algae/Presence_Absence_RedAlgae_Date.shp" %>% 
  read_sf() %>% 
  mutate(area = st_area(.)) %>% 
  group_by(date) %>% 
  reframe(area = sum (area)) %>% 
  mutate(prop = as.numeric((area/mask)*100)) %>% 
  dplyr::select(-area)

(plot <- data.frame(date = c(1952,1958),
           prop = c(0,0)) %>% 
  bind_rows(Pres_algae) %>% 
    mutate(prop = case_when(date == 2012 ~ 41.3,
                            date == 2024 ~ 41.8,
                            T ~ prop)) %>% 
ggplot(aes(x = date, y = prop))+
  # geom_bar(stat = "identity",size = 10 )+
  geom_point(size = 4)+
  xlab("Year")+
  ylab("Cover of Gracilaria (%)")+
  
  geom_line(alpha = 0.4, linetype = "dashed")+
  geom_vline(xintercept = 1971, linetype = "dashed", color = "darkred", linewidth=1)+
  geom_vline(xintercept = 1994, linetype = "dashed", color = "goldenrod", linewidth=1)+
    geom_rug(sides = "b") +  # Add ticks at the bottom for each x-value
  geom_text(
    x = 1970, 
    y = 30, 
    label = expression(paste("First introduction of ", italic("Crassostrea gigas"))), 
    angle = 90
  )+
  geom_text(
    x = 1972, 
    y = 30, 
    label = expression(paste("in South Brittany")), 
    angle = 90
  )+
  geom_text(
    x = 1993, 
    y = 31, 
    label = expression(paste("First record of ", italic("Gracilaria vermiculophylla"))), 
    angle = 90
  )+
  geom_text(
    x = 1995, 
    y = 32, 
    label = expression(paste("in the Belon Estuary")), 
    angle = 90
  )+
  theme_Bede()+
  theme(axis.text.x = element_text(size = 15),
        axis.text.y = element_text(size = 15),
        axis.title.x = element_text(size = 20),
        axis.title.y = element_text(size = 20), 
        plot.background = element_rect(fill = "white", color = NA))
)

ggsave("Paper/Figures/High_res/Cover_Gracillaria_vs_Time.png", plot, width = 10, height = 6, dpi = 400)
ggsave("Paper/Figures/Low_res/Cover_Gracillaria_vs_Time.png", plot, width = 10, height = 6, dpi = 200)

A clear shift from bare sediment to vegetated mudflats has been observed over the past 70 years, corresponding to the colonization of the Belon Estuary by *G. vermiculophylla* (<a href="#fig-HistoricalMap" class="quarto-xref">Figure 4</a>). In the 50s, the tidal flats showed no detectable presence of vegetation. In the 70s some darkening of the sediment became discerbible, but the first unambiguous presence of *G. vermiculophylla* was 1982. During the subsequent decades, the cover of algae increased and in 2024, the high-resolution mapping done with the drone showed that monospecific mats of *G. vermiculophylla* exclusively colonised the mudflat.

In [ ]:
knitr::include_graphics("./Figures/Low_res/Historical_maps.png")

From the early recordings in the 1950s through the late 1970s, *Gracilaria vermiculophylla* coverage remained effectively at 0% (<a href="#fig-HistoricalPlot" class="quarto-xref">Figure 5</a>). Shortly after the introduction of *Crassostrea gigas* in the estuary, in 1971-1972 (see vertical red dashed line in the figure), the first detectable presence of *G. vermiculophylla* emerged. By 1976, it covered 2.5% (0.7 ha) of the Pont du Guilly area, and by 1978 it had increased slightly to 3.0% (0.9 ha). From 1982 onward, coverage expanded more rapidly, increasing from 6.6% (2.0 ha) in 1982 to 14.7% (4.5 ha) in 1992 and nearly 30% (9.0 ha) by 1997. This upward trend continued into the 21st century, peaking at 41.2% (13.3 ha) in 2012. Although coverage fluctuated somewhat thereafter (40.6% in 2019 and 41.8% in 2024), it remained consistently high, indicating sustained and widespread colonization.

In [ ]:
knitr::include_graphics("./Figures/Low_res/Cover_Gracillaria_vs_Time.png")

## 4.2 Spectral description

In [ ]:
library(tidyverse)
library(terra)
library(sf)
library(signal)
library(Utilities.Package)

 # Set the path to the directory containing the CSV files
csv_directory <- "Data/ASD"

# List all CSV files in the directory
csv_files <- list.files(path = csv_directory, pattern = "\\.csv$", full.names = TRUE)

# Read and combine all CSV files into one dataframe
df <- csv_files %>%
  lapply(read.csv) %>%  # Read each CSV file into a list of dataframes
  bind_rows() %>%         # Combine all dataframes into one
  dplyr::filter(Letter == "Field") %>% 
  dplyr::select(c(Wavelength, mean_RAW, ID)) %>% 
  pivot_wider(names_from = ID, values_from = mean_RAW)



# User-defined parameters
p <- 3      # polynomial order for Savitzky-Golay
n <- 11     # frame length (odd number, e.g. 11)
factor <- 5 # degrade resolution by a factor of 5

# Assuming your initial dataframe is called df
# df: first column is wavelength, subsequent columns are spectra
wavelength <- df[[1]]
reflectances <- df[,-1]

# Compute original wavelength spacing
d_lambda <- wavelength[2] - wavelength[1]

# 1. Degrade the spectral resolution
# Create a new wavelength vector with coarser spacing
new_wavelength <- seq(from = min(wavelength), to = max(wavelength), by = factor * d_lambda)

# Interpolate the spectra onto the new wavelength grid
degraded_spectra <- apply(reflectances, 2, function(spec) {
  approx(x = wavelength, y = spec, xout = new_wavelength)$y
})

# Now our working wavelength and reflectances become:
wavelength <- new_wavelength
reflectances <- degraded_spectra

# Update the spacing to reflect the new degraded resolution
d_lambda <- wavelength[2] - wavelength[1]

# 2. Perform initial Savitzky-Golay smoothing on each degraded-resolution spectrum
smooth_spectra <- apply(reflectances, 2, sgolayfilt, p = p, n = n)

# Define a function for first derivative using central differences
first_derivative <- function(x, d) {
  n <- length(x)
  d1 <- numeric(n)
  d1[2:(n-1)] <- (x[3:n] - x[1:(n-2)]) / (2*d)
  d1[1] <- NA
  d1[n] <- NA
  return(d1)
}

# 3. Compute the first derivative from the smoothed spectra
d1_spectra <- apply(smooth_spectra, 2, first_derivative, d = d_lambda)

# Helper function to safely smooth a vector that may have NAs at boundaries
safe_sgolay <- function(x, p, n) {
  # Replace NAs at edges with nearest non-NA values before smoothing
  if (is.na(x[1])) {
    first_valid <- which(!is.na(x))[1]
    x[1:(first_valid-1)] <- x[first_valid]
  }
  if (is.na(x[length(x)])) {
    last_valid <- rev(which(!is.na(x)))[1]
    x[(last_valid+1):length(x)] <- x[last_valid]
  }
  sgolayfilt(x, p = p, n = n)
}

# 4. Smooth the first derivative again using Savitzky-Golay
d1_spectra_smooth <- apply(d1_spectra, 2, safe_sgolay, p = p, n = n)

# Define a function for second derivative using central differences
second_derivative <- function(x, d) {
  n <- length(x)
  d2 <- numeric(n)
  d2[2:(n-1)] <- (x[3:n] - 2*x[2:(n-1)] + x[1:(n-2)]) / (d^2)
  d2[1] <- NA
  d2[n] <- NA
  return(d2)
}

# 5. Compute the second derivative from the smoothed first derivative
d2_spectra <- apply(d1_spectra_smooth, 2, second_derivative, d = d_lambda)

# 6. Finally, smooth the second derivative again using Savitzky-Golay
d2_spectra_smooth <- apply(d2_spectra, 2, safe_sgolay, p = p, n = n)

# Combine results into a final dataframe
result_df <- data.frame(wavelength = wavelength, d2_spectra_smooth) %>% 
  as_tibble() %>% 
  pivot_longer(-wavelength, names_to = "spectra", values_to = "deriv") %>% 
  group_by(wavelength) %>% 
  reframe(mean = mean(deriv, na.rm = T),
          sd = sd(deriv, na.rm=T))


df_spectra <- csv_files %>%
  lapply(read.csv) %>%  # Read each CSV file into a list of dataframes
  bind_rows() %>%         # Combine all dataframes into one
  dplyr::filter(Letter == "Field")%>% 
  group_by(Wavelength) %>% 
  reframe(mean = mean(mean_RAW, na.rm = T),
          sd = sd(mean_RAW, na.rm=T))

chla= 678
PE = 565
PC = 620

a <- ggplot(result_df) +
  geom_line(aes(x = wavelength+10, y = mean))+
  geom_ribbon(aes(x = wavelength+10, y = mean, ymin = mean-sd, ymax = mean+sd), alpha = 0.05)+
  geom_vline(xintercept = chla,
    color = "darkred", linetype = "dashed")+ # chl-a
  geom_vline(xintercept = PE,
    color = "darkgreen", linetype = "dashed")+ 
  geom_vline(xintercept = PC,
    color = "goldenrod", linetype = "dashed")+
  # geom_vline(xintercept = 645, linetype = "dashed")+

  xlim(400, 700)+
  theme_Bede()+
    geom_text(x= 420, y = 10e-06, label = "B", size = 8)+
  ylab("Second Derivative")+
  xlab("Wavelength")+
  # geom_text(
  #   x = 557, 
  #   y = -0.000006, 
  #   label = "Absorption Phycoerythrin", 
  #   angle = 90,
  #   color = "darkgreen"
  # )+
  # geom_text(
  #   x = 612, 
  #   y = -0.000006, 
  #   label = "Absorption Phycocyanin", 
  #   angle = 90,
  #   color = "goldenrod"
  # )+
  # geom_text(
  #   x = 670, 
  #   y = -0.000006, 
  #   label = "Absorption Chlorophyll-a", 
  #   angle = 90,
  #   color = "darkred"
  # )+
  scale_y_continuous(position = "right",
                     labels = scales::label_number(
                       suffix = "e-06", scale = 1e6))+
  theme(panel.grid.major = ggplot2::element_line(linetype = "dotted", 
      colour = alpha("grey30",0.5), linewidth = 0.25),
      plot.background = element_rect(fill = "white", color = NA)
  )


b <- ggplot(df_spectra) +
  geom_line(aes(x = Wavelength, y = mean))+
  geom_ribbon(aes(x = Wavelength, y = mean, ymin = mean-sd, ymax = mean+sd), alpha = 0.05)+
  geom_vline(xintercept = chla,
    color = "darkred", linetype = "dashed")+ # chl-a
  geom_vline(xintercept = PE,
    color = "darkgreen", linetype = "dashed")+ 
  geom_vline(xintercept = PC,
    color = "goldenrod", linetype = "dashed")+
  # geom_vline(xintercept = 645, linetype = "dashed")+
  ylab("Reflectance")+
  geom_text(
    x = 552, 
    y = 0.17, 
    label = "Absorption Phycoerythrin", 
    angle = 90,
    color = "darkgreen"
  )+
  geom_text(
    x = 607, 
    y = 0.17, 
    label = "Absorption Phycocyanin", 
    angle = 90,
    color = "goldenrod"
  )+
  geom_text(
    x = 665, 
    y = 0.17, 
    label = "Absorption Chlorophyll-a", 
    angle = 90,
    color = "darkred"
  ) +
    geom_text(x= 420, y = 0.232, label = "A", size = 8)+

  theme_Bede()+
    theme( panel.grid.major = ggplot2::element_line(linetype = "dotted", 
      colour = alpha("grey30",0.5), linewidth = 0.25),
      plot.background = element_rect(fill = "white", color = NA))

plot <- (b)+(a)
plot

 ggsave("Paper/Figures/High_res/plot_spectral_signature.png",plot, width = 10, height = 5, dpi = 400)
 ggsave("Paper/Figures/Low_res/plot_spectral_signature.png",plot, width = 10, height = 5, dpi = 200)

##### INDEX ######
# 
# spec_lib <- readxl::read_excel("Data/speclib/Spectral_library_VEG_smoothed_A_GERAUD.xlsx") %>% 
#   pivot_longer(-Wavelength, names_to = "Species", values_to = "Ref") %>% 
#   mutate(ID = Species,
#          Species = sub("\\.\\.\\..*$", "", Species), 
#          Class = case_when(str_detect(Species, "nodosum") ~ "Phaeophyceae",
#                            str_detect(Species, "paludosa") ~ "Bacillariophyceae",
#                            str_detect(Species, "spiralis") ~ "Phaeophyceae",
#                            str_detect(Species, "limosum") ~ "Bacillariophyceae",
#                            str_detect(Species, "ramosissima") ~ "?",
#                            str_detect(Species, "Ulva ") ~ "Chlorophyceae",
#                            str_detect(Species, "serratus") ~ "Phaeophyceae",
#                            str_detect(Species, "Codium ") ~ "Chlorophyceae",
#                            str_detect(Species, "vesiculosus") ~ "Phaeophyceae",
#                            str_detect(Species, "okamurae") ~ "Phaeophyceae",
#                            str_detect(Species, "Vaucheria") ~ "Xanthophyceae",
#                            str_detect(Species, "marina") ~ "Magnoliopsida",
#                            str_detect(Species, "noltei") ~ "Magnoliopsida")) %>% 
#   group_by(Class, Wavelength) 
# 
#  
#  indices_speclib <- spec_lib %>% 
#    pivot_wider(names_from = "Wavelength", values_from = "Ref") %>% 
#    group_by(Species,ID,Class) %>% 
#    reframe(NDVI = (`800` - `668`)/(`800` + `668`),
#            GLI = ((`550` - `668`)+(`550` - `451`))/(2*`550`+`668`+`451`),
#            GNDVI =  (`800` - `550`)/(`800` + `550`),
#            EVI = (`800` - `668`)/(`800` - 6*`668` - 7.5*`451`+1),
#            # MSAVI =(2*`800`+ 1 - sqrt(((2*`800`-1)*(2*`800`-1))-8*(`800`-`668`)))/2
#            )
#  
#  
#  df_spectra_indices <- csv_files %>%
#   lapply(read.csv) %>%  # Read each CSV file into a list of dataframes
#   bind_rows() %>%         # Combine all dataframes into one
#   dplyr::filter(Letter == "Field") %>% 
#    mutate(Class = "Rhodophyceae",
#           Species = "G.vermiculophylla") %>% 
#    dplyr::select(c(Class,Species,ID,Wavelength,mean_RAW)) %>% 
#    pivot_wider(names_from = "Wavelength", values_from = "mean_RAW") %>% 
#    group_by(Species,ID,Class) %>% 
#    reframe(NDVI = (`800` - `668`)/(`800` + `668`),
#            GLI = ((`550` - `668`)+(`550` - `451`))/(2*`550`+`668`+`451`),
#            GNDVI =  (`800` - `550`)/(`800` + `550`),
#            EVI = (`800` - `668`)/(`800` - 6*`668` - 7.5*`451`+1),
#            # MSAVI =(2*`800`+ 1 - sqrt(((2*`800`-1)*(2*`800`-1))-8*(`800`-`668`)))/2
#            )
# 
#  all__indices <-df_spectra_indices %>% 
#    bind_rows(indices_speclib)
# 
#  all__indices %>% 
#    dplyr::filter(!Class %in% c("Bacillariophyceae","?")) %>% 
#    pivot_longer(c(NDVI,GLI,GNDVI,EVI), names_to = "Indice",values_to = "value") %>% 
#    ggplot(aes(x = Class, y = value, color = Class, group = interaction(Indice,Class)))+
#    facet_wrap(~ Indice)+
#    geom_boxplot()+
#    theme_Bede()
#  
#  spec_lib %>% 
#   reframe(mean = mean(Ref, na.rm = T),
#           sd = sd(Ref, na.rm = T)) %>% 
#   ggplot()+
#   geom_line(aes(x = Wavelength, y =mean, group = Class, color = Class))+
#   geom_vline(xintercept = chla,
#     color = "darkred", linetype = "dashed")+ # chl-a
#   geom_vline(xintercept = PE,
#     color = "darkgreen", linetype = "dashed")+ 
#   geom_vline(xintercept = PC,
#     color = "goldenrod", linetype = "dashed")

The spectral signature of *G. vermiculophylla* was characterized by a reflectance pattern in the visible region of the spectrum shaped by the photosynthetic and accessory pigments common to all rhodophytes (<a href="#fig-SpecDescri" class="quarto-xref">Figure 6</a> A). This pattern was primarily driven by phycoerythrin and phycocyanin, which exhibited maximum absorption peaks at approximately 565 nm and 620 nm, respectively. An additional absorption feature around 495 nm was likely attributable to accessory carotenoid pigments. The most pronounced absorption peak occurred at 675 nm, corresponding to chlorophyll-a absorption. The second derivative analysis clearly highlighted the inflection points corresponding to the main absorption peaks at 495, 565, 620, and 675 nm, allowing for more precise identification of the wavelength associated with these pigments (<a href="#fig-SpecDescri" class="quarto-xref">Figure 6</a> B).

In [ ]:
knitr::include_graphics("./Figures/Low_res/plot_spectral_signature.png")

## 4.3 Spatial distribution

In [ ]:
library(terra)
library(tidyverse)
library(Utilities.Package)
library(sf)

msk <- "Data/shp/mask/Belon_32630.shp" %>% 
  read_sf()

DISCOV_rast <- rast("Data/DISCOV/Belon_1104_MS_DISCOV_InvaSea_26112024_pred.tif") %>% 
  project(crs(msk), method = "near") %>% 
  crop(msk, mask = T)

values(DISCOV_rast)[values(DISCOV_rast) == 0] = NA
names(DISCOV_rast) <- "layer"
DISCOV_rast<-DISCOV_rast %>% 
  as.factor()

DISCOV_Belon <- ggplot()+
    tidyterra::geom_spatraster(data = DISCOV_rast, 
                               mapping = aes(fill = layer),
                               maxcell = 500832*8)+
  scale_fill_manual(breaks = c(1,2,3,4,5,6,8),
                    values=c("#DAA520","#b3ff1a","#389318","#873e23","#b3002d","#a2a3a2","#42c9bc",NA,NA),
                    na.value = NA,
                    name = "Classes",
                    labels = c("Bacillariophyceae",
                               "Chlorophyceae",
                               "Magnoliopsida",
                               "Phaeophyceae",
                               "Rhodophyceae",
                               "Sediment",
                               "Water"))+
  geom_text(aes(x =  ext(DISCOV_rast)[1]+0.04*(ext(DISCOV_rast)[2]-ext(DISCOV_rast)[1]),
                y =  ext(DISCOV_rast)[4]-0.08*(ext(DISCOV_rast)[4]-ext(DISCOV_rast)[3]),
                label = "A"),
                size = 10)+
  coord_sf(expand = F,
           xlim = c(450180.4, 451247.2+18),
           ylim = c(5296561, 5297042))+
  theme_Bede_Map()+
  # coord_equal()+
   theme(
     # axis.text.x = element_text(size = 10, angle = -20, color = NA,hjust = 0),
         axis.text.y = element_text(size = 10, vjust=-8, hjust=0.5, angle = 90),
         axis.text.x = element_text(size = 10, vjust=8, hjust=0.5),
         axis.ticks.x = element_blank(),
         axis.ticks.y = element_blank(),
         axis.title = element_blank(),
         plot.background = element_rect(fill = "white", color = NA),
         panel.background = element_blank(),  # No panel background
         legend.background = element_blank(),
         legend.position = c(0.98, 0.98),
         legend.text = element_text(size = 10),
         legend.key.size = unit(0.5, "cm"),
         plot.margin = margin(0, 0, -10, -10))

ggsave("Paper/Figures/High_res/Prediction_Map/Belon_DISCOV.png",DISCOV_Belon,width = 10, height = 4.44, dpi = 400)

In [ ]:
library(sf)
library(terra)
library(tidyverse)
library(Utilities.Package)

msk <- "Data/shp/mask/Belon_32630.shp" %>% 
  read_sf()

Belon_RGB <- "Data/RGB/Belon_RGB_MULTI.tif" %>% 
  rast() %>% 
  crop(msk, mask = T)

# values(Belon_RGB[[1]])[values(Belon_RGB[[1]]) == 0] = NA

# RGB(Belon_RGB) <- 1:3


lim_x<- c(as.numeric(ext(Belon_RGB)[1:2]))
lim_y<- c(as.numeric(ext(Belon_RGB)[3:4]))


 RGB_Belon<- ggplot()+
   tidyterra::geom_spatraster_rgb(data = Belon_RGB,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4010533
         )+
  geom_text(aes(x =  ext(Belon_RGB)[1]+0.04*(ext(Belon_RGB)[2]-ext(Belon_RGB)[1]),
                y =  ext(Belon_RGB)[4]-0.08*(ext(Belon_RGB)[4]-ext(Belon_RGB)[3]),
                label = "B"),
                size = 10)+
    ggspatial::annotation_scale(location = "br",
                                pad_y = unit(0.7, "cm"),
                                text_cex=2)+
   theme_Bede_Map()+
  coord_sf(expand = F,
           xlim = c(450180.4, 451247.2+18),
           ylim = c(5296561, 5297042))+
  # coord_equal()+
   theme(axis.text.y = element_text(size = 10, vjust=-8, hjust=0.5, angle = 90),
         axis.text.x = element_text(size = 10, vjust=8, hjust=0.5),
         axis.ticks.x = element_blank(),
         axis.ticks.y = element_blank(),
         plot.background = element_rect(fill = "white", color = NA), 
         axis.title = element_blank(),
         panel.background = element_blank(),  # No panel background
         legend.background = element_blank(),
         legend.position = c(0.98, 0.98),
         legend.text = element_text(size = 10),
         legend.key.size = unit(0.5, "cm"),
         plot.margin = margin(0, 0, -10, -10))

 
 ggsave("Paper/Figures/High_res/Prediction_Map/Belon_RGB.png",RGB_Belon,width = 10, height = 4.44, dpi = 400)

In [ ]:
library(terra)
library(tidyverse)
library(Utilities.Package)
library(scico)



msk <- "Data/shp/mask/Belon_DEM_32630.shp" %>% 
  read_sf()

DEM_rast <- rast("Data/DEM/Belon_DEM_LiDAR.tif") %>% 
  crop(msk, mask = T) %>% 
  terra::resample(Belon_RGB, method = "near")

p999 <- quantile(DEM_rast %>% as.data.frame() %>%  pull(Belon_DEM_LiDAR), probs = 0.999)

values(DEM_rast)[values(DEM_rast) > p999] = NA
values(DEM_rast)[values(DEM_rast) < 0] = NA

lim_x<- c(as.numeric(ext(Belon_RGB)[1:2]))
lim_y<- c(as.numeric(ext(Belon_RGB)[3:4]))


bathy_Belon <- ggplot()+
   # tidyterra::geom_spatraster_rgb(data = Belon_RGB,
   #       r = 1,
   #       g = 2,
   #       b = 3,
   #       maxcell = 1000,
   #       alpha = 0
   #       )+
    tidyterra::geom_spatraster(data = DEM_rast, 
                               mapping = aes(fill = Belon_DEM_LiDAR ),
                               maxcell = 500832)+
  scale_fill_scico(palette = "batlow", 
                   na.value = NA,
                   name = "Elevation",
                   breaks = c(0,1.5,3),
                   limits = c(0,3),
                   labels = c("0 m","1.5 m", "3 m")) +
  geom_text(aes(x =  ext(Belon_RGB)[1]+0.04*(ext(Belon_RGB)[2]-ext(Belon_RGB)[1]),
                y =  ext(Belon_RGB)[4]-0.08*(ext(Belon_RGB)[4]-ext(Belon_RGB)[3]),
                label = "C"),
                size = 10)+
  theme_Bede_Map()+
  coord_sf(expand = F,
           xlim = c(450180.4, 451247.2+18),
           ylim = c(5296561, 5297042))+
  # coord_equal()+
   theme(axis.text.y = element_text(size = 10, vjust=-8, hjust=0.5, angle = 90),
         axis.text.x = element_text(size = 10, vjust=8, hjust=0.5),
         axis.ticks.x = element_blank(),
         axis.ticks.y = element_blank(),
         plot.background = element_rect(fill = "white", color = NA), 
         axis.title = element_blank(),
         panel.background = element_blank(),  # No panel background
         legend.background = element_blank(),
         legend.position = c(0.98, 0.98),
         legend.text = element_text(size = 10),
         legend.key.size = unit(0.5, "cm"),
         plot.margin = margin(0, 0, -10, -10))

 ggsave("Paper/Figures/High_res/Prediction_Map/Belon_Bathy.png",bathy_Belon,width = 10, height = 4.44, dpi = 400)
 
 # nrow(DEM_rast %>%  as.data.frame() %>% dplyr::filter(!is.na(Belon_DEM_LiDAR)))

In [ ]:
msk <- "Data/shp/mask/Belon_DEM_32630.shp" %>% 
  read_sf()

Belon_RGB <- "Data/RGB/Belon_RGB_MULTI.tif" %>% 
  rast() %>% 
  crop(msk, mask = T)

DEM_rast <- rast("Data/DEM/Belon_DEM_LiDAR.tif") %>% 
  crop(msk, mask = T)

Angle_df<-read_csv("Data/Model/FullDF_10timesResample_DEMSlope_PredPercent.csv") %>% 
  dplyr::filter(Site  == "Belon")

angle_rast <- Angle_df %>% 
  dplyr::select(x,y,Slope_Cat) %>% 
  mutate(Slope_Cat = case_when(Slope_Cat == "Slope" ~ 1,
                               Slope_Cat == "Flat" ~ 2,
                               Slope_Cat == "Vertical" ~ 3)) %>% 
  terra::rast(type="xyz", crs="EPSG:32630") %>% 
  crop(msk, mask = T) %>%
  as.factor()
  

angle_plot <- ggplot()+
  #   tidyterra::geom_spatraster(data = DEM_rast, 
  #                              mapping = aes(fill = Belon_DEM_LiDAR ),
  #                              maxcell = 100,
  #                              alpha=0, 
  #                              show.legend = F)+
  # scico::scale_fill_scico(palette = "batlow", na.value = NA,
  #                  name = "Bathymetry",
  #                  breaks = c(0,1.5,3),
  #                  limits = c(0,3),
  #                  labels = c("0 m","1.5 m", "3 m"))+
  # ggnewscale::new_scale_fill()+
  # geom_tile(Angle_df, mapping = aes(x = x, y = y, fill = Slope_Cat)) +
  tidyterra::geom_spatraster(data = angle_rast, 
                               mapping = aes(fill = Slope_Cat  ),
                               maxcell = 1310*581,
                               # alpha=0, 
                               show.legend = T)+
  scale_fill_manual(breaks = c(2,1,3),
                    values=c("#8E7CC3", "#76A5AF", "#93C47D"),
                    na.value = NA,
                    name = "Angle",
                    labels = c("Flat",
                               "Angled",
                               "Steep"))+
  # geom_sf(data = msk, fill = "white", color = "white",show.legend = F)+
  geom_text(aes(x =  ext(Belon_RGB)[1]+0.04*(ext(Belon_RGB)[2]-ext(Belon_RGB)[1]),
                y =  ext(Belon_RGB)[4]-0.08*(ext(Belon_RGB)[4]-ext(Belon_RGB)[3]),
                label = "D"),
                size = 10)+
  theme_Bede_Map()+
  coord_sf(expand = F,
           xlim = c(450180.4, 451247.2+18),
           ylim = c(5296561, 5297042))+
  # coord_equal()+
   theme(axis.text.y = element_text(size = 10, vjust=-8, hjust=0.5, angle = 90),
         axis.text.x = element_text(size = 10, vjust=8, hjust=0.5),
         axis.ticks.x = element_blank(),
         axis.ticks.y = element_blank(),
         plot.background = element_rect(fill = "white", color = NA), 
         axis.title = element_blank(),
         panel.background = element_blank(),  # No panel background
         legend.background = element_blank(),
         legend.position = c(0.98, 0.98),
         legend.text = element_text(size = 10),
         legend.key.size = unit(0.5, "cm"),
         plot.margin = margin(0, 0, -10, -10))
 
ggsave("Paper/Figures/High_res/Prediction_Map/Belon_angle.png",angle_plot,width = 10, height = 4.44, dpi = 400)

In [ ]:
library(sf)
library(terra)
library(tidyverse)
library(Utilities.Package)

RGB <- rast("Paper/Figures/High_res/Prediction_Map/Belon_RGB.png")
RGB(RGB) <- 1:3

Pred <- rast("Paper/Figures/High_res/Prediction_Map/Belon_DISCOV.png")
RGB(Pred) <- 1:3


Bathy <- rast("Paper/Figures/High_res/Prediction_Map/Belon_Bathy.png")
RGB(Pred) <- 1:3

Angle <- rast("Paper/Figures/High_res/Prediction_Map/Belon_angle.png")
RGB(Angle) <- 1:3

offset = 50

ext(Pred) <- c(0,4000,1689-offset,1689*2-offset)
ext(RGB) <- c(0,4000,0,1689)
ext(Bathy) <- c(0,4000,-1689+(offset),offset)
ext(Angle) <- c(0,4000,-1689+(2*offset)-1823,-1689+2*offset)


 Belon_patchwork<- ggplot()+
   tidyterra::geom_spatraster_rgb(data = Pred,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   tidyterra::geom_spatraster_rgb(data = RGB,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   tidyterra::geom_spatraster_rgb(data = Bathy,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   tidyterra::geom_spatraster_rgb(data = Angle,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1823
         )+
   coord_equal()+
   theme_void()

ggsave("Paper/Figures/High_res/Prediction_Map/Belon_maps.png",Belon_patchwork,width = 7.89, height = 10, dpi = 400)
ggsave("Paper/Figures/Low_res/Belon_maps.png",Belon_patchwork,width = 7.89, height = 10, dpi = 200)

In [ ]:
library(sf)
library(terra)
library(tidyverse)
library(Utilities.Package)

RGB <- rast("Paper/Figures/High_res/Prediction_Map/Belon_RGB.png")
RGB(RGB) <- 1:3

Pred <- rast("Paper/Figures/High_res/Prediction_Map/Belon_DISCOV.png")
RGB(Pred) <- 1:3


Bathy <- rast("Paper/Figures/High_res/Prediction_Map/Belon_Bathy.png")
RGB(Pred) <- 1:3

Angle <- rast("Paper/Figures/High_res/Prediction_Map/Belon_angle.png")
RGB(Angle) <- 1:3

offset = 50

ext(RGB) <- c(4000,8000,0,1776)
ext(Pred) <- c(0,4000,0,1776)
ext(Bathy) <- c(0,4000,-1776,0)
ext(Angle) <- c(4000,8000,-1776,0)


 Belon_patchwork<- ggplot()+
   tidyterra::geom_spatraster_rgb(data = Pred,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   tidyterra::geom_spatraster_rgb(data = RGB,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   tidyterra::geom_spatraster_rgb(data = Bathy,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   tidyterra::geom_spatraster_rgb(data = Angle,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1823
         )+
   coord_equal()+
   theme_void()

ggsave("Paper/Figures/High_res/Prediction_Map/Belon_maps.png",Belon_patchwork,width = 10, height = 4.44, dpi = 400)
ggsave("Paper/Figures/Low_res/Belon_maps.png",Belon_patchwork,width = 10, height = 10, dpi = 200)

In [ ]:
library(sf)
library(terra)
library(tidyverse)
library(Utilities.Package)

RGB <- rast("Paper/Figures/High_res/Prediction_Map/Belon_RGB.png")
RGB(RGB) <- 1:3

Pred <- rast("Paper/Figures/High_res/Prediction_Map/Belon_DISCOV.png")
RGB(Pred) <- 1:3


Bathy <- rast("Paper/Figures/High_res/Prediction_Map/Belon_Bathy.png")
RGB(Pred) <- 1:3

Angle <- rast("Paper/Figures/High_res/Prediction_Map/Belon_angle.png")
RGB(Angle) <- 1:3

offset = 50

ext(RGB) <- c(0,4000,-1776,0)
ext(Pred) <- c(0,4000,0,1776)
ext(Bathy) <- c(0,4000,-1776*2,-1776)
ext(Angle) <- c(0,4000,-1776*3,-1776*2)


 Belon_patchwork<- ggplot()+
   tidyterra::geom_spatraster_rgb(data = Pred,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   tidyterra::geom_spatraster_rgb(data = RGB,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   tidyterra::geom_spatraster_rgb(data = Bathy,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   tidyterra::geom_spatraster_rgb(data = Angle,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1823
         )+
   coord_equal()+
   theme_void()

ggsave("Paper/Figures/High_res/Prediction_Map/Belon_maps.png",Belon_patchwork,width = 5.63, height =10, dpi = 400)
ggsave("Paper/Figures/Low_res/Belon_maps.png",Belon_patchwork,width = 5.63, height = 10, dpi = 200)

In [ ]:
library(sf)
library(terra)
library(tidyverse)

# file_list <- "Data/Shiny_Validation" %>% 
#   list.files(".shp",recursive = T, full.names = T) %>% 
#   as_tibble() %>% 
#   rename(path = "value") %>% 
#   mutate(filename = gsub(".*/","",path) %>% 
#            gsub("shp","",.),
#          User_date = sapply(strsplit(path, "/"), function(x) x[3]))
# 
# for(i in 1:nrow(file_list)){
#   shp_i <- read_sf(file_list$path[i]) %>% 
#     mutate(user_date = file_list$User_date[i],
#            snapshot =  file_list$filename[i])
#   
#   if(i == 1){
#     shp <- shp_i
#   }else{
#     shp <- rbind(shp,shp_i)
#   }
# }

# write_sf(shp,"Data/Shiny_Validation/shp_all_validation.shp")

shp <- read_sf("Data/Shiny_Validation/shp_all_validation.shp") %>% 
  mutate(site = sapply(strsplit(snapshot, "_"), function(x) x[1]))

img_list <- "Data/DISCOV" %>% 
  list.files(pattern = ".tif", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(filename = gsub(".*/","",path),
         site = case_when(str_detect(filename,"Aven") ~ "Aven",
                          str_detect(filename,"Belon") ~ "Belon",
                          str_detect(filename,"North") ~ "SajaNorth",
                          str_detect(filename,"South") ~ "SajaSouth"))


for (i in 1:nrow(img_list)) {
  
  img <- rast(img_list$path[i])
    # as.factor()
  names(img) = "pred"
  
  shp_i <- shp %>% 
    dplyr::filter(site == unique(img_list$site)[i]) %>% 
    mutate(Truth = case_when(Vegetation == "Red algae" ~ 1,
                             T ~ 0)) %>% 
    rasterize(img, field = "Truth")
  names(shp_i) = "true"

  stk <- c(img,shp_i) %>% 
    as.data.frame(xy = T) %>% 
    dplyr::filter(!is.na(true)) %>% 
    mutate(pred = case_when(pred == 5 ~ 1, 
                            T ~ 0),
           site = unique(img_list$site)[i])
  
  if (i==1){
    validation_df <- stk
  }else{
    validation_df <- rbind(validation_df,stk)
  }
}

library(caret)

a <- confusionMatrix(as.factor(validation_df$pred),as.factor(validation_df$true))

validation_matrix <- a$overall %>% 
  as.data.frame() %>% 
  rownames_to_column() %>% 
  bind_rows({
     a$byClass %>% 
      as.data.frame() %>% 
      rownames_to_column()
  })

names(validation_matrix) <- c("metric", "value")
write.csv(validation_matrix, "Data/validation_metrics.csv", row.names = F)

In [ ]:
library(tidyverse)
library(sf)

df_validation <- 
  "../Data/validation_metrics.csv" %>%
  # "Data/validation_metrics.csv" %>%
  read.csv()

The classification map obtained from the neural network algorithm is shown for the Belon estuary (<a href="#fig-Belon" class="quarto-xref">Figure 7</a> A). Among the main classes of the intertidal vegetation, the class fo Rhodophyceae (red macroalgae, in red) was the dominant algal cover, forming extensive, continous patches colonizing almost the entire mudflat. In contrast, Bacillariophyceae (diatoms biofilm, in orange) and Chlorophyceae (Green macroalgae, in green) exhibited more localized distributions, typically restricted to smaller, fragmented patches. A few Phaeophyceae (brown macroalgae, in brown) were confined to limited patches in the upper intertidal attached to rocks. In the Saja esturay, Rhodophyceae cover was more scarsed, due to a strong Chlorophyceae presence on this site (Annexe D: <a href="#sec-AnnexeD" class="quarto-xref">Section 7.4</a>).

Across the study sites the presence/absence of *G. vermiculophylla* was classified with a global accuracy of 91.1 %, a sensitivity of 96.5 % and a specificity of 71.5 %.

The elevation map showed that the main mats of G. vermiculophylla were between 1 and 2 m above mean sea level (<a href="#fig-Belon" class="quarto-xref">Figure 7</a> C). Algal presence was markedly elevation-driven, with lower intertidal zones closer to the tidal channel consistently exibiting reduced macroalgal cover. Most of the intertidal flats exibited slope below 10° (Violet, (<a href="#fig-Belon" class="quarto-xref">Figure 7</a> D). Angled surfaces (10° \< Slope \< 40°) often found next to tidal channels were exibiting almost no vegetation cover.

In [ ]:
knitr::include_graphics("./Figures/Low_res/Belon_maps.png")

In [ ]:
library(tidyverse)
library(Utilities.Package)
library(mgcv)
library(brms)
library(tidybayes)
library(cowplot)


Full_df<-read_csv("Data/Model/FullDF_10timesResample_DEMSlope_PredPercent.csv")

m_Red <- brm(bf(Pred ~ DEM+Slope_Cat+ (1|Site)),
         data = Full_df, family = Beta(), cores = 8, iter = 1000,
         warmup = 100, control = list(adapt_delta = 0.9999),
         save_pars = save_pars(latent = TRUE),
         file = "Data/Model/Pred_Beta_agg_1000_DEM_SlopeCat_reSite_resampled")


NewData_1<-expand_grid(DEM=seq(min(Full_df$DEM),max(Full_df$DEM),length.out=100),
                       Slope_Cat=factor(c("Flat",
                            "Slope",
                            "Vertical")),
                       Site=unique(Full_df$Site))

NewData1_brms<-NewData_1%>% 
  add_linpred_draws(m_Red, ndraws=2000,seed=123,transform=TRUE)

Total<-NewData1_brms%>% 
  group_by(DEM)%>% 
  summarise(median_qi(.linpred,.width=c(0.89))) %>% 
  expand_grid(Slope_Cat=factor(c("Flat",
                            "Slope",
                            "Vertical")))

# (plot <- NewData1_brms%>% 
#   group_by(DEM,Slope_Cat)%>% 
#   summarise(median_qi(.linpred,.width=c(0.89)))%>%
#   ggplot(aes(x = DEM,y=y)) +
#   geom_ribbon(aes(ymax = ymax,ymin=ymin,color=Slope_Cat),
#               fill = NA,
#               linetype = "dashed") +
#   geom_line(aes(colour=Slope_Cat),
#             linewidth=1)+
#   geom_line(data = Total,aes(x=DEM,y=y),colour="black")+
#   scale_fill_manual(name="Slope",values=c("#8E7CC3","#76A5AF","#93C47D"),
#                     labels=c("Flat",
#                             "Angled",
#                             "Vertical"))+
#   scale_colour_manual(name="Slope",values=c("#8E7CC3","#76A5AF","#93C47D"),
#                     labels=c("Flat",
#                             "Angled",
#                             "Vertical"))+
#   scale_y_continuous(breaks = c(0.1,0.2,0.3,0.4,0.5),
#                      labels = c("10","20","30","40","50"))+
#   labs(x="Bathymetry (m)",y="Intertidal Area Covered\n by Gracilaria vermiculophylla (%)")+
#   facet_wrap(~Slope_Cat,ncol =2)+
#   theme_Bede()+
#   theme(strip.text = element_blank(),
#         legend.position= c(0.75,0.25), 
#         legend.title = element_text(size = 20, margin = margin(b = 20)),
#         legend.text = element_text(size  = 15),
#         legend.key.size = unit(1,"cm"),
#         legend.spacing.y = unit(1, "cm"),
#         axis.text.x = element_text(size = 15),
#         axis.text.y = element_text(size = 15),
#         axis.title.x = element_text(size = 20),
#         axis.title.y = element_text(size = 20), 
#         plot.background = element_rect(fill = "white", color = NA))+
#   guides(fill = guide_legend(nrow = 1),
#          colour = guide_legend(nrow = 1))
#   )

(plot <- NewData1_brms%>% 
  group_by(DEM,Slope_Cat)%>% 
  summarise(median_qi(.linpred,.width=c(0.89)))%>%
  ggplot(aes(x = DEM,y=y)) +
  geom_ribbon(aes(ymax = ymax,ymin=ymin,color=Slope_Cat),
              fill = NA,
              linetype = "dashed") +
  geom_line(aes(colour=Slope_Cat),
            linewidth=1)+
  geom_line(data = Total,aes(x=DEM,y=y),colour="black")+
  scale_fill_manual(name="Slope",values=c("#8E7CC3","#76A5AF","#93C47D"),
                    labels=c("Flat",
                            "Angled",
                            "Steep"))+
  scale_colour_manual(name="Slope",values=c("#8E7CC3","#76A5AF","#93C47D"),
                    labels=c("Flat",
                            "Angled",
                            "Steep"))+
  scale_y_continuous(breaks = c(0.1,0.2,0.3,0.4,0.5),
                     labels = c("10","20","30","40","50"))+
  labs(x="Elevation (m)",y="Cover of Gracilaria vermiculophylla (%)")+
  facet_wrap(~Slope_Cat,ncol =3)+
  theme_Bede()+
  theme(strip.text = element_blank(),
        # legend.title = element_text(size = 17),
        legend.title = element_blank(),
        legend.text = element_text(size  = 15),
        legend.key.spacing.x = unit(5,"cm"),
        legend.position = "top",
        legend.key.size = unit(0.7,"cm"),
        # legend.key.spacing.y = unit(-10, "cm"),
        axis.text.x = element_text(size = 15),
        axis.text.y = element_text(size = 15),
        axis.title.x = element_text(size = 17),
        axis.title.y = element_text(size = 17), 
        plot.background = element_rect(fill = "white", color = NA))
  )

NewData1_brms%>% 
  group_by(DEM,Slope_Cat)%>% 
  summarise(median_qi(.linpred,.width=c(0.89))) %>% 
  group_by(Slope_Cat) %>% 
  reframe(min = min(y),
          max = max(y))

ggsave("Paper/Figures/High_res/GAM_slope_cover.png", width = 10, height = 5, dpi = 400)
ggsave("Paper/Figures/Low_res/GAM_slope_cover.png", width = 10, height = 5, dpi = 200)

Overall, the percent cover of *G. vermiculophylla* increased with elevation, as shown by the general relationship (<a href="#fig-Gam_Slope" class="quarto-xref">Figure 8</a>, black line), which rises from approximately 16% at the lowest elevation to about 30% at the highest elevation. This indicates a consistent positive association between elevation and algal cover.

When accounting for slope, the flatter the slope, the higher the percent cover of *G. vermiculophylla*. For flat slopes, the cover ranged from approximately 20% at the lowest elevation to nearly 38% at the highest elevation. In contrast, the increase was less pronounced for angled slopes, ranging from around 16% to 32%. The cover was the lowest on steep slopes, starting at about 15% and rising only slightly above 30% at the highest elevation (<a href="#fig-Gam_Slope" class="quarto-xref">Figure 8</a>). This demonstrates that slope modifies the relationship, with flatter slopes supporting a greater percent cover of the algae.

In [ ]:
knitr::include_graphics("./Figures/Low_res/GAM_slope_cover.png")

# 5. Discussion

## 5.1 Drone mapping *G. vermiculophylla* with machine learning

In this study, we produced the first spatial distribution maps of the invasive red alga *Gracilaria vermiculophylla* using a multispectral drone survey conducted at low tide in Atlantic estuaries representing varied environmental conditions. In southern Brittany, the species formed monospecific mats, while in the Cantabrian region of Spain, it was intermixed with other intertidal vegetation. Distinguishing among these vegetation types was a key prerequisite for the analysis.

To achieve this, we adapted the deep learning-based classification model DISCOV ([Oiry et al., 2024](#ref-rs16234383)), initially developed to discriminate seagrass from green macroalgae. Although the original model included Rhodophyceae as a class, this group constituted less than 3% of its training dataset. In contrast, the updated model presented here was trained on a dataset in which *G. vermiculophylla* covered 26 % of approximately one million pixels. This improved dataset allowed the model to achieve an accuracy of 91.1 %.

Rhodophytes possess unique phycobilin pigments, enabling their spectral distinction from other macroalgal groups ([Douay et al., 2022](#ref-rs14020346); [Mcilwaine et al., 2019](#ref-rs11060704); [Olmedo-Masat et al., 2020](#ref-olmedo2020far)). Even with the ten-band multispectral sensor used in our study, it remained feasible to discriminate the major classes of intertidal macrophytes ([Davies et al., 2023](#ref-davies2023multi); [Oiry et al., 2024](#ref-rs16234383); [Román et al., 2021](#ref-roman2021using)). However, the model identifies *G. vermiculophylla* at the class level (Rhodophyceae) rather than at the species level. Although hyperspectral approaches may allow finer taxonomic resolution ([Douay et al., 2022](#ref-rs14020346); [Olmedo-Masat et al., 2020](#ref-olmedo2020far)), it is unlikely that Gracilaria species can be precisely distinguished using standard multispectral sensors.

Ecological factors also aid in differentiating *G. vermiculophylla*. Unlike many other macroalgae that require hard substrates, *G. vermiculophylla* establishes itself on soft-bottom sediments. In fact, it is commonly found on mudflats, anchoring its thalli in the top 10 cm of mud ([Surget, 2017](#ref-surget2017processus)), and inhabits the upper intertidal zone—an unusual trait for a Rhodophyte ([Abreu et al., 2011](#ref-abreu2011nitrogen); [Davoult et al., 2017](#ref-davoult2017multiple)). By reliably detecting *G. vermiculophylla* in these soft-substrate, upper intertidal habitats, our method provides a framework for identifying environmental conditions that favor its spread, potentially offering managers early-warning indicators to control its expansion before it reaches nuisance levels. Thus, combining spectral data with sediment characteristics provides a strong indicator of *G. vermiculophylla* presence in European Atlantic estuaries, complementing the physical variables already used in species distribution modeling ([Mendoza-Segura et al., 2023](#ref-jmse11020367)).

In addition, the scalability of drone-based surveying facilitates repeat mapping to detect temporal shifts in the distribution and abundance of *G. vermiculophylla.* Such continuous monitoring could capture seasonal patterns of colonization, allowing researchers and environmental managers to evaluate the effectiveness of mitigation measures, track long-term ecological impacts, and anticipate future shifts in habitat suitability under changing climate conditions.

## 5.2 *G. vermiculophylla* spatial distribution and mudflat topography

The spatial distribution of *G. vermiculophylla* across intertidal zones reveals a distinct relationship with mudflat topography, which significantly influences algal density and coverage. Our results show that higher elevations within the intertidal zone support greater densities of *G. vermiculophylla*. A pattern that aligns with findings by Thomsen et al. ([2009](#ref-thomsen2009distribution)), where elevated areas provided optimal conditions for algal survival. *G. vermiculophylla* demonstrates remarkable physiological plasticity, enabling it to tolerate a broad spectrum of environmental conditions, including temperature fluctuations ([Sotka et al., 2018](#ref-sotka2018combining)), nutrient variability ([Abreu et al., 2011](#ref-abreu2011nitrogen)), and a wide range of salinities ([Weinberger et al., 2008](#ref-weinberger2008invasive)). Its capacity for sustained growth under low salinity conditions ([Nyberg, 2007](#ref-nyberg2007introduced); [Rueness, 2005](#ref-rueness2005life)) underpins its successful establishment and persistence within polyhaline and mesohaline estuarine environments. A strong constrain, however, for the algae is the hydrodynamism. Unlike seagrasses, another type of marine plant that can also colonize soft sediment, which possess rhizomes that provide robust anchorage, *G. vermiculophylla* lacks such specialized structures. Its attachment to the substrate relies solely on its buried thalli, which embed into the upper layers of soft sediment. This mode of anchorage renders the alga particularly vulnerable to high hydrodynamic conditions, as it lacks the structural stability needed to withstand strong currents or wave action. To partially bury its thalli into the sediment, *G. vermiculophylla* requires areas with high sedimentation rates. These conditions are typically found in the upper regions of estuarine systems or in proximity to other macrophytes, such as *Sporobolus* spp. (commonly known as small cordgrass), which reduce current velocity and promote sediment deposition ([Mudd et al., 2010](#ref-mudd2010does)). This observation aligns with the findings illustrated in <a href="#fig-HistoricalMap" class="quarto-xref">Figure 4</a>, which show that one of the first areas colonized by *G. vermiculophylla* in 1992 in the Bélon estuary, France, was located near a patch of salt marsh.  
As a result, *G. vermiculophylla* compensates for its vulnerability to hydrodynamic forces by forming dense mats, which enhance its stability and facilitate its persistence and proliferation in intertidal and estuarine environments with low to moderate hydrodynamic conditions ([Surget, 2017](#ref-surget2017processus)).

The negative relationship between slope steepness and the density of *G. vermiculophylla* can be explained by the physical and ecological characteristics of steeper mudflat areas. Steeper slopes are typically associated with higher rates of water runoff during tidal exchanges, resulting in stronger hydrodynamic forces. These forces can lead to increased sediment erosion, reduced sedimentation, and less stable substrate conditions, which are unfavorable for *G. vermiculophylla* to anchor its thalli effectively ([Besterman et al., 2021](#ref-besterman2021predicting)). Furthermore, steeper slopes may limit the retention of organic matter and nutrients, reducing the availability of essential resources needed for algal growth. In contrast, flatter areas within the intertidal zone are more likely to accumulate fine sediments and retain water for longer durations during low tides, creating a more stable and nutrient-rich environment conducive to *G. vermiculophylla* proliferation. Additionally, these conditions may favor the formation of dense algal mats, which further stabilize the sediment and promote growth.

## 5.3 Monitoring *Gracilaria vermiculophylla* Invasion Dynamics

The invasive red alga *G. vermiculophylla* represents a significant example of delayed recognition and documentation in biological invasions. Historical aerial imagery and photo-interpretation analyses from the Bélon Estuary suggest the initial presence of this species in 1976 <a href="#fig-HistoricalMap" class="quarto-xref">Figure 4</a>, preceding its first formal description in European waters in 1996 by two decades ([Rueness, 2005](#ref-rueness2005life)). This delay likely stems from insufficient early monitoring frameworks and limited awareness of its ecological impacts, which often characterize the early stages of invasive species colonization. It also arises from the fact that others red macroalgae species, resembling *G. vermiculophylla* and native to this area (e.g. *Gracilaria gracilis*) were already present at sites where *G. vermiculophylla* was introduced, further complicating its detection. This lag highlights challenges associated with the detection, monitoring, and reporting of invasive species and their ecological impacts during early colonization.

The appearance of *G. vermiculophylla* in 1976 corresponds with the introduction of the Pacific oyster (*Crassostrea gigas*) into the estuary, a few years before, between 1971 and 1975, and a potential vector for algal dispersal through aquaculture activities ([Grizel and Heral, 1991](#ref-grizel1991introduction); [Rueness, 2005](#ref-rueness2005life)). Aquaculture practices, such as the transfer of oyster spat and equipment between regions, facilitate the unintentional transport of invasive algal fragments. For instance, *G. vermiculophylla* may have attached to shells or nets used in oyster farming, enabling its spread to new estuarine habitats. After initial establishment, the alga progressively occupied suitable habitats, consistent with theoretical invasion dynamics involving a lag phase followed by rapid spread ([Arim et al., 2006](#ref-arim2006spread)). The establishment of *G. vermiculophylla* likely induced changes in sediment characteristics, trophic interactions, and habitat structure prior to formal recognition ([BenDor and Metcalf, 2006](#ref-bendor2006spatial)). Such shifts are comparable to documented impacts in similar systems ([Crowl et al., 2008](#ref-crowl2008spread); [Gallardo et al., 2016](#ref-gallardo2016global)), yet remain difficult to quantify without early monitoring data. Remote sensing using multispectral drone mapping can provide high-resolution, spatially explicit data, but it must be combined with repeated, *in situ* field measurements to maximize its potential ([Chadwick et al., 2020](#ref-chadwick2020integrating); [Zoffoli et al., 2023](#ref-zoffoli2023remote)). Temporal repetition makes it possible to assess dynamic processes, and integrating these mapping approaches with *in situ* analyses of local infauna, carbon cycling, riverine inputs, and sedimentology yields valuable data for local managers. Such an integrated approach can determine how the invasive algae affects the local ecosystem and, more broadly, forecast its potential impact on other estuarine environments facing similar invasion events.

The temporal gap between the first presence and documentation reflects limitations in early surveillance, potentially underestimating ecological and economic impacts during the initial colonization phase. Studies on invasion dynamics demonstrate that early detection is crucial for effective containment and management, particularly before an invasion reaches the exponential spread phase, which complicates control efforts ([Arim et al., 2006](#ref-arim2006spread); [BenDor and Metcalf, 2006](#ref-bendor2006spatial); [Elton, 2020](#ref-elton2020ecology)). Specific practices, such as the removal of early-stage algal mats, implementation of physical barriers to prevent further spread, and public awareness campaigns, could mitigate the impacts during this critical phase ([Green and Grosholz, 2021](#ref-green2021functional); [Jones et al., 2021](#ref-jones2021use); [Simberloff, 2021](#ref-simberloff2021maintenance)). In the Bélon Estuary, *G. vermiculophylla* appears to have thrived under ecological conditions favorable to its proliferation, enabling the formation of dense mats in about 6 years (between 1976 and 1982; <a href="#fig-HistoricalMap" class="quarto-xref">Figure 4</a>) after its first detection in the estuary. This undocumented growth likely contributed to substantial changes in the estuarine ecosystem. Historical aerial imagery has provided valuable insights into long-term invasion patterns by enabling the retrospective identification of shifts in habitat characteristics. Modern drone-based systems enhance this capacity through high spatial and temporal resolution, enabling the rapid detection of invasive species at early stages of establishment. By capturing detailed data on the spatial distribution and habitat preferences of species such as *G. vermiculophylla*, remote sensing facilitates timely interventions, allowing stakeholders to take rapid measures to limit the invasion. Integrating these tools into routine monitoring programs offers a scalable and efficient means to track invasive species dynamics and inform targeted management strategies, such as habitat restoration, removal of invasive mats, and prevention of further spread through targeted interventions. Expanding these methodologies to lower-cost RGB-based detection would further democratize access to monitoring tools, enabling more widespread application for early detection and rapid response. These tools could also be integrated into community-driven management programs, empowering local stakeholders to monitor invasive species and implement timely control measures.

# 6. Conclusion

In this study, we demonstrated the potential of high-resolution drone-based multispectral remote sensing to map the spatial and temporal distribution of the invasive red macroalga *G. vermiculophylla* in European estuaries. By employing the DISCOV model, updated to include an extensive dataset of Rhodophyceae pixels, we achieved a classification accuracy of 91.1%. Our analysis revealed a clear spatial relationship between *G. vermiculophylla* and intertidal topography retrieved from LiDAR, with its cover consistently higher in flat, elevated mudflats compared to lower and steeper areas. The temporal progression, derived from a historical dataset spanning over seven decades, highlights the progressive establishment and expansion of the algae. Notably, our remote sensing analysis confirmed the presence of *G. vermiculophylla* in the Bélon Estuary approximately 20 years before its first scientific description, emphasizing the value of retrospective mapping.

The historical analysis of aerial imagery provided crucial insights into the dynamics of *G. vermiculophylla*’s invasion, revealing a lag phase followed by rapid colonization. This expansion coincided with the development of oyster aquaculture, suggesting a potential link between human activities and the proliferation of this invasive species. The remarkable physiological plasticity of *G. vermiculophylla*, enabling it to thrive in diverse environmental conditions, further underscores its adaptability and invasive potential. However, its reliance on sediment stability and vulnerability to hydrodynamic forces delineate its preferred habitats within intertidal zones.

These findings underscore the crucial role of remote sensing in ecological research, particularly in studying invasive species. By leveraging high-resolution, scalable technologies, we can not only map current distributions but also uncover historical patterns that would otherwise remain unknown. The identification of *G. vermiculophylla* decades prior to its formal description exemplifies this capability. This discovery provides a compelling basis for re-evaluating historical data to understand the broader implications of invasive species dynamics. Moving forward, integrating hyperspectral sensors could enhance species-level discrimination, while adopting low-cost RGB-based methods could extend monitoring capacities to a broader range of stakeholders. Incorporating these advancements into environmental management frameworks can enable proactive monitoring, timely interventions, and habitat restoration efforts. These advancements will be crucial for informing management strategies, fostering community engagement, and preserving estuarine biodiversity in the face of ongoing ecological changes.



# 7. Annexes

## 7.1 Annexes A - Updated training dataset

In [ ]:
library(tidyverse)
library(flextable)


Attachement du package : 'flextable'

L'objet suivant est masqué depuis 'package:purrr':

    compose



## 7.2 Annexes B - Validation dataset

In [ ]:
library(tidyverse)
library(flextable)


df_validation <- 
  

df_flx_valid <- "../Data/Shiny_Validation/shp_all_validation.shp" %>%
  # "Data/Shiny_Validation/shp_all_validation.shp" %>%
  read_sf() %>% 
  mutate(user = sapply(strsplit(user_date, "_"), function(x) x[1]),
         site = sapply(strsplit(snapshot, "_"), function(x) x[1]),
         site = case_when(site == "Aven" ~ "Notre-Dame De Tremor",
                          site == "Belon" ~ "Pont de Guilly",
                          site == "SajaNorth" ~ "Marisma de Cudón",
                          site == "SajaSouth" ~ "Marisma de Cortiguera"),
         PresenceAbs = case_when(Vegetation == "Red algae" ~ "Present",
                                 T ~ "Absent")) %>% 
  rename(Site = "site") 

total <- df_flx_valid %>% 
  group_by(PresenceAbs) %>% 
  reframe(n = n()) %>% 
  pivot_wider(names_from = PresenceAbs, values_from = n) %>% 
  mutate(Site = "Total")

table_data <- df_flx_valid %>% 
  group_by(Site, PresenceAbs) %>% 
  reframe(n = n()) %>% 
  pivot_wider(names_from = "PresenceAbs", values_from = "n") %>% 
  rbind(total) %>% 
  rowwise() %>% 
  mutate(Total = sum(c(Absent, Present)))

# Get the number of rows for the table data
n_rows <- nrow(table_data)

# Create the flextable with a horizontal line
table_data %>%
  flextable() %>%
  align(align = "center", part = "all") %>% 
  set_table_properties(layout = "autofit") %>%
  hline(i = n_rows - 1, border = officer::fp_border(color = "black", width = 1))



## 7.3 Annexes C - List of historical images records

In [ ]:
library(flextable)

 data.frame(
  Date = c(
    "1952-04-26",
    "1958-04-22",
    "1976-07-? ",
    "1978-08-22",
    "1982-08-11",
    "1992-05-17",
    "1997-04-11",
    "2012-07-24",
    "2024-04-11"),
  Type = c(
    "Black and White",
    "Black and White",
    "Black and White",
    "Black and White",
    "Black and White",
    "True Color",
    "Black and White",
    "True Color",
    "True Color"),
  DataSource = c(    
    "IGN",
    "IGN",
    "IGN",
    "IGN",
    "IGN",
    "IGN",
    "IGN",
    "IGN",
    "Drone Flight"),
  Res = c(
     "10",
     "90",
     "4",
     "44",
     "44",
     "70",
     "64",
     "18",
     "3"
  )) %>% 
 dplyr::rename(`Resolution (cm per Pixel)` = "Res",
               `Data Source` = "DataSource") %>% 
  flextable()%>%
  align(align = "center", part = "all") %>% # Align text to center
  set_table_properties(layout = "autofit")



## 7.4 Annexes D - Maps of the Saja esturay, Spain

In [ ]:
library(terra)
library(tidyverse)
library(Utilities.Package)
library(sf)

msk <- "Data/shp/mask/SajaSouth_32630.shp" %>% 
  read_sf()

DISCOV_rast <- rast("Data/DISCOV/Saja_South_25062024_DISCOV_InvaSea_26112024_pred.tif") %>% 
  project(crs(msk), method = "near") %>% 
  crop(msk, mask = T)

values(DISCOV_rast)[values(DISCOV_rast) == 0] = NA
names(DISCOV_rast) <- "layer"
DISCOV_rast<-DISCOV_rast %>% 
  as.factor()

DISCOV_Saja <- ggplot()+
    tidyterra::geom_spatraster(data = DISCOV_rast, 
                               mapping = aes(fill = layer),
                               maxcell = 500832*8)+
  scale_fill_manual(breaks = c(1,2,3,4,5,6,8),
                    values=c("#DAA520","#b3ff1a","#389318","#873e23","#b3002d","#a2a3a2","#42c9bc",NA,NA),
                    na.value = NA,
                    name = "DISCOV v2",
                    labels = c("Bacillariophyceae",
                               "Chlorophyceae",
                               "Magnoliopsida",
                               "Phaeophyceae",
                               "Rhodophyceae",
                               "Sediment",
                               "Water"))+
  geom_text(aes(x =  ext(DISCOV_rast)[1]+0.04*(ext(DISCOV_rast)[2]-ext(DISCOV_rast)[1]),
                y =  ext(DISCOV_rast)[4]-0.08*(ext(DISCOV_rast)[4]-ext(DISCOV_rast)[3]),
                label = "A"),
                size = 10)+
  coord_sf(expand = F,
           xlim = ext(msk)[1:2],
           ylim = ext(msk)[3:4])+
  theme_Bede_Map()+
  # coord_equal()+
   theme(
     # axis.text.x = element_text(size = 10, angle = -20, color = NA,hjust = 0),
         axis.text.y = element_text(size = 15),
         axis.text.x = element_text(size = 15),
         # axis.ticks.x = element_blank(),
         axis.title = element_blank(),
         plot.background = element_rect(fill = "white", color = NA),
         legend.background = element_blank(),
         legend.position = c(0.98,0.98),
         legend.text = element_text(size = 15),
         legend.key.size = unit(1,"cm"))

ggsave("Paper/Figures/High_res/Prediction_Map/Saja_DISCOV.png",DISCOV_Saja,width = 10, height = 9, dpi = 400)

In [ ]:
library(sf)
library(terra)
library(tidyverse)
library(Utilities.Package)

msk <- "Data/shp/mask/SajaSouth_32630.shp" %>% 
  read_sf()

Saja_RGB <- "Data/RGB/SajaSouth_RGB_LIDAR2.tif" %>% 
  rast() %>% 
  crop(msk, mask = T)

# values(Belon_RGB[[1]])[values(Belon_RGB[[1]]) == 0] = NA

# RGB(Belon_RGB) <- 1:3


lim_x<- c(as.numeric(ext(Saja_RGB)[1:2]))
lim_y<- c(as.numeric(ext(Saja_RGB)[3:4]))

NAflag(Saja_RGB) <- 0

RGB_Saja<- ggplot()+
   tidyterra::geom_spatraster_rgb(data = Saja_RGB,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4010533,
         max_col_value = 255
         )+
  geom_text(aes(x =  ext(Saja_RGB)[1]+0.04*(ext(Saja_RGB)[2]-ext(Saja_RGB)[1]),
                y =  ext(Saja_RGB)[4]-0.08*(ext(Saja_RGB)[4]-ext(Saja_RGB)[3]),
                label = "B"),
                size = 10)+
    ggspatial::annotation_scale(location = "br",
                                pad_y = unit(0.5, "cm"),
                                text_cex=2)+
   theme_Bede_Map()+
  coord_sf(expand = F,
           xlim = ext(msk)[1:2],
           ylim = ext(msk)[3:4])+
  # coord_equal()+
   theme(
     # axis.text.x = element_text(size = 10, angle = -20, color = NA,hjust = 0),
         axis.text.y = element_text(size = 15),
         axis.text.x = element_text(size = 15),
         # axis.ticks.x = element_blank(),
         axis.title = element_blank(),
         plot.background = element_rect(fill = "white", color = NA),
         legend.background = element_blank(),
         legend.position = c(0.98,0.98),
         legend.text = element_text(size = 15),
         legend.key.size = unit(1,"cm"))

 
 ggsave("Paper/Figures/High_res/Prediction_Map/Saja_RGB.png",RGB_Saja,width = 10, height = 9, dpi = 400)

In [ ]:
library(sf)
library(terra)
library(tidyverse)
library(Utilities.Package)

RGB <- rast("Paper/Figures/High_res/Prediction_Map/Saja_RGB.png")
RGB(RGB) <- 1:3

Pred <- rast("Paper/Figures/High_res/Prediction_Map/Saja_DISCOV.png")
RGB(Pred) <- 1:3

pic <- rast("Data/RGB/PictureSaja.png")
RGB(pic) <- 1:3


plot(pic)

offset = 50

ext(Pred) <- c(0,4000,0,3600)
ext(RGB) <- c(4000,8000,0,3600)
ext(pic) <- c(2000,6000,3600,3600 + (1868))


Saja_patchwork<- ggplot()+
   tidyterra::geom_spatraster_rgb(data = Pred,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*3600
         )+
   tidyterra::geom_spatraster_rgb(data = RGB,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*3600
         )+
   tidyterra::geom_spatraster_rgb(data = pic,
         r = 1,
         g = 2,
         b = 3,
         maxcell = 4000*1689
         )+
   coord_equal()+
   theme_void()

ggsave("Paper/Figures/High_res/Prediction_Map/Saja_maps.png",Saja_patchwork,width = 10, height = 6.83, dpi = 400)
ggsave("Paper/Figures/Low_res/Saja_maps.png",Saja_patchwork,width = 10, height =  6.83, dpi = 200)

In [ ]:
knitr::include_graphics("./Figures/Low_res/Saja_maps.png")



# 8. References

Abreu, M.H., Pereira, R., Buschmann, A., Sousa-Pinto, I., Yarish, C., 2011. Nitrogen uptake responses of gracilaria vermiculophylla (ohmi) papenfuss under combined and single addition of nitrate and ammonium. Journal of Experimental Marine Biology and Ecology 407, 190–199.

Agisoft, 2019. [Agisoft metashape](https://www.agisoft.com/).

Arim, M., Abades, S.R., Neill, P.E., Lima, M., Marquet, P.A., 2006. Spread dynamics of invasive species. Proceedings of the National Academy of Sciences 103, 374–378.

BenDor, T.K., Metcalf, S.S., 2006. The spatial dynamics of invasive species spread. System Dynamics Review: The Journal of the System Dynamics Society 22, 27–50.

Besterman, A.F., McGlathery, K.J., Reidenbach, M.A., Wiberg, P.L., Pace, M.L., 2021. Predicting benthic macroalgal abundance in shallow coastal lagoons from geomorphology and hydrologic flow patterns. Limnology and Oceanography 66, 123–140.

Blanchet, H., Gouillieux, B., Alizier, S., others, 2014. Multiscale patterns in the diversity and organization of benthic intertidal fauna among french atlantic estuaries. Journal of Sea Research 90, 95–110. <https://doi.org/10.1016/j.seares.2014.02.014>

Brunier, G., Tamura, T., Anthony, E.J., Dussouillez, P., Gardel, A., 2022. Evolution of the french guiana coast from late pleistocene to holocene based on chenier and beach sand dating. Regional Environmental Change 22, 122.

Bürkner, P.-C., 2021. Bayesian item response modeling in R with <span class="nocase">brms</span> and Stan. Journal of Statistical Software 100, 1–54. <https://doi.org/10.18637/jss.v100.i05>

Bürkner, P.-C., 2018. Advanced Bayesian multilevel modeling with the R package <span class="nocase">brms</span>. The R Journal 10, 395–411. <https://doi.org/10.32614/RJ-2018-017>

Bürkner, P.-C., 2017. <span class="nocase">brms</span>: An R package for Bayesian multilevel models using Stan. Journal of Statistical Software 80, 1–28. <https://doi.org/10.18637/jss.v080.i01>

Calleja, F., Galván, C., Silió-Calzada, A., Juanes, J.A., Ondiviela, B., 2017. Long-term analysis of zostera noltei: A retrospective approach for understanding seagrasses’ dynamics. Marine environmental research 130, 93–105.

Castaing, P., Guilcher, A., 1995. Morphosedimentary evolution of ria-type estuaries. Earth Surface Processes and Landforms 20, 361–376. <https://doi.org/10.1002/esp.3290200408>

Chadwick, K.D., Brodrick, P.G., Grant, K., Goulden, T., Henderson, A., Falco, N., Wainwright, H., Williams, K.H., Bill, M., Breckheimer, I., others, 2020. Integrating airborne remote sensing and field campaigns for ecology and earth system science. Methods in Ecology and Evolution 11, 1492–1508.

Chand, S., Bollard, B., 2021. Low altitude spatial assessment and monitoring of intertidal seagrass meadows beyond the visible spectrum using a remotely piloted aircraft system. Estuarine, Coastal and Shelf Science 255, 107299.

Chang, W., Cheng, J., Allaire, J., Sievert, C., Schloerke, B., Xie, Y., Allen, J., McPherson, J., Dipert, A., Borges, B., 2024. [Shiny: Web application framework for r](https://CRAN.R-project.org/package=shiny).

Crowl, T.A., Crist, T.O., Parmenter, R.R., Belovsky, G., Lugo, A.E., 2008. The spread of invasive species and infectious disease as drivers of ecosystem change. Frontiers in Ecology and the Environment 6, 238–246.

Davies, B.F.R., Gernez, P., Geraud, A., Oiry, S., Rosa, P., Zoffoli, M.L., Barillé, L., 2023. Multi-and hyperspectral classification of soft-bottom intertidal vegetation using a spectral library for coastal biodiversity remote sensing. Remote Sensing of Environment 290, 113554.

Davies, B.F.R., Oiry, S., Rosa, P., Zoffoli, M.L., Sousa, A.I., Thomas, O.R., Smale, D.A., Austen, M.C., Biermann, L., Attrill, M.J., others, 2024b. A sentinel watching over inter-tidal seagrass phenology across western europe and north africa. Communications Earth & Environment 5, 382.

Davies, B.F.R., Oiry, S., Rosa, P., Zoffoli, M.L., Sousa, A.I., Thomas, O.R., Smale, D.A., Austen, M.C., Biermann, L., Attrill, M.J., others, 2024a. Intertidal seagrass extent from sentinel-2 time-series show distinct trajectories in western europe. Remote Sensing of Environment 312, 114340.

Davoult, D., Surget, G., Stiger-Pouvreau, V., Noisette, F., Riera, P., Stagnol, D., Androuin, T., Poupart, N., 2017. Multiple effects of a gracilaria vermiculophylla invasion on estuarine mudflat functioning and diversity. Marine Environmental Research 131, 227–235.

Diruit, W., Le Bris, A., Bajjouk, T., Richier, S., Helias, M., Burel, T., Lennon, M., Guyot, A., Ar Gall, E., 2022. Seaweed habitats on the shore: Characterization through hyperspectral UAV imagery and field sampling. Remote Sensing 14. <https://doi.org/10.3390/rs14133124>

Douay, F., Verpoorter, C., Duong, G., Spilmont, N., Gevaert, F., 2022. New hyperspectral procedure to discriminate intertidal macroalgae. Remote Sensing 14. <https://doi.org/10.3390/rs14020346>

Douglas, T.J., Coops, N.C., Drever, M.C., Hunt, B.P., Martin, T.G., 2024. Linking microphytobenthos distribution and mudflat geomorphology under varying sedimentary regimes using unoccupied aerial vehicle (UAV)-acquired multispectral reflectance and photogrammetry. Science of The Total Environment 173675.

Duffy, J.P., Pratt, L., Anderson, K., Land, P.E., Shutler, J.D., 2018. Spatial assessment of intertidal seagrass meadows using optical imaging systems and a lightweight drone. Estuarine, Coastal and Shelf Science 200, 169–180.

Elton, C.S., 2020. The ecology of invasions by animals and plants. Springer Nature.

Firth, L.B., Foggo, A., Watts, T., Knights, A.M., DeAmicis, S., 2024. Invasive macroalgae in native seagrass beds: Vectors of spread and impacts. Annals of Botany 133, 41–50.

Gallardo, B., Clavero, M., Sánchez, M.I., Vilà, M., 2016. Global ecological impacts of invasive species in aquatic ecosystems. Global change biology 22, 151–163.

Ginneken, V. van, Vries, E. de, others, 2018. The global dispersal of the non-endemic invasive red alga gracilaria vermiculophylla in the ecosystems of the euro-asia coastal waters including the wadden sea unesco world heritage coastal area: Awful or awesome? Oceanography & Fisheries Open Access Journal 8, 4–26.

Green, S.J., Grosholz, E.D., 2021. Functional eradication as a framework for invasive species control. Frontiers in Ecology and the Environment 19, 98–107.

Grizel, H., Heral, M., 1991. Introduction into france of the japanese oyster (crassostrea gigas). ICES Journal of Marine Science 47, 399–403.

Gurgel, C.F.D., Norris, J.N., Schmidt, W.E., Le, H.N., Fredericq, S., 2018. Systematics of the gracilariales (rhodophyta) including new subfamilies, tribes, subgenera, and two new genera, agarophyton gen. Nov. And crassa gen. nov. Phytotaxa 374, 1–23.

Hijmans, R.J., 2024. [Terra: Spatial data analysis](https://CRAN.R-project.org/package=terra).

IGN, 2024. Remonter le temps.

Jones, P.E., Tummers, J.S., Galib, S.M., Woodford, D.J., Hume, J.B., Silva, L.G., Braga, R.R., Garcia de Leaniz, C., Vitule, J.R., Herder, J.E., others, 2021. The use of barriers to limit the spread of aquatic invasive animal species: A global review. Frontiers in Ecology and Evolution 9, 611631.

Krueger-Hadfield, S.A., Kollars, N.M., Strand, A.E., Byers, J.E., Shainker, S.J., Terada, R., Greig, T.W., Hammann, M., Murray, D.C., Weinberger, F., others, 2017. Genetic identification of source and likely vector of a widespread marine invader. Ecology and evolution 7, 4432–4447.

Massé, C., Viard, F., Humbert, S., Antajan, E., Auby, I., Bachelet, G., Bernard, G., Bouchet, V.M.P., Burel, T., Dauvin, J.-C., Delegrange, A., Derrien-Courtel, S., Droual, G., Gouillieux, B., Goulletquer, P., Guérin, L., Janson, A.-L., Jourde, J., Labrune, C., Lavesque, N., Leclerc, J.-C., Le Duff, M., Le Garrec, V., Noël, P., Nowaczyk, A., Pergent-Martini, C., Pezy, J.-P., Raoux, A., Raybaud, V., Ruitton, S., Sauriau, P.-G., Spilmont, N., Thibault, D., Vincent, D., Curd, A., 2023. An overview of marine non-indigenous species found in three contrasting biogeographic metropolitan french regions: Insights on distribution, origins and pathways of introduction. Diversity 15. <https://doi.org/10.3390/d15020161>

Mcilwaine, B., Casado, M.R., Leinster, P., 2019. Using 1st derivative reflectance signatures within a remote sensing framework to identify macroalgae in marine environments. Remote Sensing 11. <https://doi.org/10.3390/rs11060704>

Mendoza-Segura, C., Fernández, E., Beca-Carretero, P., 2023. Predicted changes in the biogeographical range of gracilaria vermiculophylla under present and future climate scenarios. Journal of Marine Science and Engineering 11. <https://doi.org/10.3390/jmse11020367>

Michel, G., Le Bot, S., Lesourd, S., Lafite, R., 2021. Morpho-sedimentological and dynamic patterns in a ria type estuary: The belon estuary (south brittany, france). Journal of Maps 17, 389–400. <https://doi.org/10.1080/17445647.2021.1925170>

Mudd, S.M., D’Alpaos, A., Morris, J.T., 2010. How does vegetation affect sedimentation on tidal marshes? Investigating particle capture and hydrodynamic controls on biologically mediated sedimentation. Journal of Geophysical Research: Earth Surface 115.

Nebel, S., Beege, M., Schneider, S., Rey, G.D., 2020. A review of photogrammetry and photorealistic 3D models in education from a psychological perspective, in: Frontiers in Education. Frontiers Media SA, p. 144.

Nyberg, C.D., 2007. Introduced marine macroalgae and habitat modifiers: Their ecological role and significant attributes. Department of Marine Ecology.

Nyberg, C.D., Thomsen, M.S., Wallentinus, I., 2009. Flora and fauna associated with the introduced red alga gracilaria vermiculophylla. European Journal of Phycology 44, 395–403.

OHMI, H., 1956. CONTRIBUTIONS TO THE KNOWLEDGE OF GRACILARIACEAE FROM JAPAN: Ⅱ. On a new species of the genus gracilariopsis, with some considerations on its ecology. 北海道大學水産學部研究彙報 6, 271–279.

Oiry, S., Davies, B.F.R., Sousa, A.I., Rosa, P., Zoffoli, M.L., Brunier, G., Gernez, P., Barillé, L., 2024. Discriminating seagrasses from green macroalgae in european intertidal areas using high-resolution multispectral drone imagery. Remote Sensing 16. <https://doi.org/10.3390/rs16234383>

Olmedo-Masat, O.M., Raffo, M.P., Rodrı́guez-Pérez, D., Arijón, M., Sánchez-Carnero, N., 2020. How far can we classify macroalgae remotely? An example using a new spectral library of species from the south west atlantic (argentine patagonia). Remote Sensing 12, 3870.

Ortega, T., Ponce, R., Forja, J., Gómez-Parra, A., 2005. Fluxes of dissolved inorganic carbon in three estuarine systems of the cantabrian sea (north of spain). Journal of Marine Systems 53, 125–142.

Papenfuss, G.F., 1967. [Notes on algal nomenclature - v. Various chlorophyceae and rhodophyceae](https://marinespecies.org/aphia.php?p=sourcedetails&id=303450). Phykos 5, 95–105.

Peidro-Devesa, M.J., Martı́nez-Movilla, A., Rodrı́guez-Somoza, J.L., Sánchez, J.M., Román, M., 2024. Quantifying intertidal macroalgae stocks in the NW iberian peninsula using unmanned aerial vehicle (UAV) multispectral imagery. Regional Studies in Marine Science 103621.

Ramus, A.P., Silliman, B.R., Thomsen, M.S., Long, Z.T., 2017. An invasive foundation species enhances multifunctionality in a coastal ecosystem. Proceedings of the national academy of sciences 114, 8580–8585.

Roca, M., Dunbar, M.B., Román, A., Caballero, I., Zoffoli, M.L., Gernez, P., Navarro, G., 2022. Monitoring the marine invasive alien species rugulopteryx okamurae using unmanned aerial vehicles and satellites. Frontiers in Marine Science 9, 1004012.

Román, A., Oiry, S., Davies, B.F., Rosa, P., Gernez, P., Tovar-Sánchez, A., Navarro, G., Méléder, V., Barillé, L., 2024. Mapping intertidal microphytobenthic biomass with very high-resolution remote sensing imagery in an estuarine system. Science of The Total Environment 177025.

Román, A., Tovar-Sánchez, A., Olivé, I., Navarro, G., 2021. Using a UAV-mounted multispectral camera for the monitoring of marine macrophytes. Frontiers in Marine Science 8, 722698.

Romero, M., Andrés, A., Alonso, R., Viguri, J., Rincón, J.M., 2008. Sintering behaviour of ceramic bodies from contaminated marine sediments. Ceramics International 34, 1917–1924.

Rueness, J., 2005. Life history and molecular sequences of gracilaria vermiculophylla (gracilariales, rhodophyta), a new introduction to european waters. Phycologia 44, 120–128.

Savitzky, A., Golay, M.J., 1964. Smoothing and differentiation of data by simplified least squares procedures. Analytical chemistry 36, 1627–1639.

Sfriso, A., Wolf, M.A., Maistro, S., Sciuto, K., Moro, I., 2012. Spreading and autoecology of the invasive species gracilaria vermiculophylla (gracilariales, rhodophyta) in the lagoons of the north-western adriatic sea (mediterranean sea, italy). Estuarine, Coastal and Shelf Science 114, 192–198.

Simberloff, D., 2021. Maintenance management and eradication of established aquatic invaders. Hydrobiologia 848, 2399–2420.

Simon, O., 2024. [Shiny app for validation dataset building](https://oirysimon.shinyapps.io/shiny_validate/).

Sotka, E.E., Baumgardner, A.W., Bippus, P.M., Destombe, C., Duermit, E.A., Endo, H., Flanagan, B.A., Kamiya, M., Lees, L.E., Murren, C.J., others, 2018. Combining niche shift and population genetic analyses predicts rapid phenotypic evolution during invasion. Evolutionary Applications 11, 781–793.

Surget, G., 2017. Processus adaptatifs des végétaux marins face au changement climatique <span class="nocase">à</span> différentes <span class="nocase">é</span>chelles de temps et d’espace: Dynamique de populations, métabolomique, <span class="nocase">é</span>cophysiologie et potentiels de valorisation (PhD thesis). Université de Bretagne occidentale-Brest.

Tankoua, O.F., Buffet, P.-E., Amiard, J.-C., Amiard-Triquet, C., Mouneyrac, C., Berthet, B., 2011. Potential influence of confounding factors (size, salinity) on biomarkers in the sentinel species scrobicularia plana used in programmes monitoring estuarine quality. Environmental Science and Pollution Research 18, 1253–1263. <https://doi.org/10.1007/s11356-011-0479-3>

Terada, R., Yamamoto, H., 2002. Review of gracilaria vermiculophylla (ohmi) papenfuss and other species in japan and asia. Taxonomy of economic seaweeds, with special reference to Pacific species 8, 225–230.

Thomsen, M.S., McGlathery, K., Schwarzschild, A., Silliman, B., 2009. Distribution and ecological role of the non-native macroalga gracilaria vermiculophylla in virginia salt marshes. Biological Invasions 11, 2303–2316.

Thomsen, M.S., Staehr, P.A., Nyberg, C.D., Schwærter, S., Krause-Jensen, D., Silliman, B.R., 2007. Gracilaria vermiculophylla (ohmi) papenfuss, 1967 (rhodophyta, gracilariaceae) in northern europe, with emphasis on danish conditions, and what to expect in the future. Aquatic invasions 2, 83–94.

Thomsen, M.S., Stæhr, P.A., Nejrup, L., Schiel, D.R., 2013. Effects of the invasive macroalgae gracilaria vermiculophylla on two co-occurring foundation species and associated invertebrates. Aquatic Invasions 8, 133–145.

Valle, M., Pala, V., Lafon, V., Dehouck, A., Garmendia, J.M., Borja, A., Chust, G., 2015. Mapping estuarine habitats using airborne hyperspectral imagery, with special focus on seagrass meadows. Estuarine, Coastal and Shelf Science 164, 433–442.

Van Katwijk, M., 2003. Reintroduction of eelgrass (zostera marina l.) in the dutch wadden sea: A research overview and management vision, in: Challenges to the Wadden Sea Area. In: Proceedings of the 10th International Scientific Wadden Sea Symposium, Groningen, the Netherlands. pp. 173–195.

Weinberger, F., Buchholz, B., Karez, R., Wahl, M., 2008. The invasive red alga gracilaria vermiculophylla in the baltic sea: Adaptation to brackish water may compensate for light limitation. Aquatic Biology 3, 251–264.

Williams, S.L., Smith, J.E., 2007. A global review of the distribution, taxonomy, and impacts of introduced seaweeds. Annu. Rev. Ecol. Evol. Syst. 38, 327–359.

Zenetos, A., Tsiamis, K., Galanidi, M., Carvalho, N., Bartilotti, C., Canning-Clode, J., Castriota, L., Chainho, P., Comas-González, R., Costa, A.C., Dragičević, B., Dulčić, J., Faasse, M., Florin, A.-B., Gittenberger, A., Jakobsen, H., Jelmert, A., Kerckhof, F., Lehtiniemi, M., Livi, S., Lundgreen, K., Macic, V., Massé, C., Mavrič, B., Naddafi, R., Orlando-Bonaca, M., Petovic, S., Png-Gonzalez, L., Carbonell Quetglas, A., Ribeiro, R.S., Cidade, T., Smolders, S., Stæhr, P.A.U., Viard, F., Outinen, O., 2022. Status and trends in the rate of introduction of marine non-indigenous species in european seas. Diversity 14. <https://doi.org/10.3390/d14121077>

Zoffoli, M.L., Gernez, P., Godet, L., Peters, S., Oiry, S., Barillé, L., 2021. Decadal increase in the ecological status of a north-atlantic intertidal seagrass meadow observed with multi-mission satellite time-series. Ecological Indicators 130, 108033.

Zoffoli, M.L., Gernez, P., Oiry, S., Godet, L., Dalloyau, S., Davies, B.F.R., Barillé, L., 2023. Remote sensing in seagrass ecology: Coupled dynamics between migratory herbivorous birds and intertidal meadows observed by satellite during four decades. Remote Sensing in Ecology and Conservation 9, 420–433.